## CHICAGO CAR ACCIDENTS

#### OVERVIEW

This project aims to create a predictive model and provide insights into the primary contributory cause of vehicle accidents in Chicago based on data about the vehicle, passengers, and external factors like road conditions, weather, etc.

#### BUSINESS UNDERSTANDING

On an average span of four days, Chicago can record up to over a thousand car accidents. When you include drivers, passengers, pedestrians and cyclists, up to two thousand people can be effected. Forty-five percent of the people will experience a minor to fatal injury.
If the cause for an incident is known, a city can then plan appropriately as to what measures should be taken to prevent them from happening again.

#### DATA

The data comes from the Chicago Data Portal, maintained by the city of Chicago. The dataset contains all traffic crashes that were reported by the police within the city limits. With the crash dataset, two datasets corresponding to Vehicles and Persons involved in the crash are attached. Each crash incident has a unique crash record ID and report number, which allows for cross-referencing.

#### METHODOLOGY

Due to the many categorical features that can influence functionality, this project investigates features and their effects through different types of models in an attempt to best predict the primary contributory cause of accidents. This is done through the use of logistic regression, decision trees, random forest and gradient boosting.

#### IMPORT LIBRARIES & DATA

In [1]:
# Importing the neccessary libraries 
# Import relevant Python modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.model_selection import cross_val_predict, cross_val_score, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# Classification Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn import tree
from scipy import stats
from imblearn.over_sampling import SMOTE 
from imblearn.pipeline import Pipeline as imbpipe
from sklearn import preprocessing
from sklearn.utils import class_weight
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

# Scalers
from sklearn.impute import SimpleImputer

from sklearn.impute import KNNImputer

from sklearn.preprocessing import StandardScaler, LabelBinarizer

from sklearn.preprocessing import OneHotEncoder

In [2]:
def percentage_null(df):

    return df.isnull().sum() / len(df) * 100

In [3]:
# Importing the csv files
crash = pd.read_csv('/kaggle/input/chicago-car-crashes-files/Traffic_Crashes_-_Crashes.csv')
people = pd.read_csv('/kaggle/input/chicago-car-crashes-files/Traffic_Crashes_-_People.csv')
vehicles = pd.read_csv('/kaggle/input/chicago-car-crashes-files/Traffic_Crashes_-_Vehicles.csv')

In [4]:
crash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879740 entries, 0 to 879739
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                879740 non-null  object 
 1   CRASH_DATE_EST_I               65200 non-null   object 
 2   CRASH_DATE                     879740 non-null  object 
 3   POSTED_SPEED_LIMIT             879740 non-null  int64  
 4   TRAFFIC_CONTROL_DEVICE         879740 non-null  object 
 5   DEVICE_CONDITION               879740 non-null  object 
 6   WEATHER_CONDITION              879740 non-null  object 
 7   LIGHTING_CONDITION             879740 non-null  object 
 8   FIRST_CRASH_TYPE               879740 non-null  object 
 9   TRAFFICWAY_TYPE                879740 non-null  object 
 10  LANE_CNT                       199017 non-null  float64
 11  ALIGNMENT                      879740 non-null  object 
 12  ROADWAY_SURFACE_COND          

In [5]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1932026 entries, 0 to 1932025
Data columns (total 29 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   PERSON_ID              object 
 1   PERSON_TYPE            object 
 2   CRASH_RECORD_ID        object 
 3   VEHICLE_ID             float64
 4   CRASH_DATE             object 
 5   SEAT_NO                float64
 6   CITY                   object 
 7   STATE                  object 
 8   ZIPCODE                object 
 9   SEX                    object 
 10  AGE                    float64
 11  DRIVERS_LICENSE_STATE  object 
 12  DRIVERS_LICENSE_CLASS  object 
 13  SAFETY_EQUIPMENT       object 
 14  AIRBAG_DEPLOYED        object 
 15  EJECTION               object 
 16  INJURY_CLASSIFICATION  object 
 17  HOSPITAL               object 
 18  EMS_AGENCY             object 
 19  EMS_RUN_NO             object 
 20  DRIVER_ACTION          object 
 21  DRIVER_VISION          object 
 22  PHYSICAL_CONDITION

In [6]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1794597 entries, 0 to 1794596
Data columns (total 71 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   CRASH_UNIT_ID             int64  
 1   CRASH_RECORD_ID           object 
 2   CRASH_DATE                object 
 3   UNIT_NO                   int64  
 4   UNIT_TYPE                 object 
 5   NUM_PASSENGERS            float64
 6   VEHICLE_ID                float64
 7   CMRC_VEH_I                object 
 8   MAKE                      object 
 9   MODEL                     object 
 10  LIC_PLATE_STATE           object 
 11  VEHICLE_YEAR              float64
 12  VEHICLE_DEFECT            object 
 13  VEHICLE_TYPE              object 
 14  VEHICLE_USE               object 
 15  TRAVEL_DIRECTION          object 
 16  MANEUVER                  object 
 17  TOWED_I                   object 
 18  FIRE_I                    object 
 19  OCCUPANT_CNT              float64
 20  EXCEED_SPEED_LIMIT_I    

#### DATA CLEANING AND EXPLORATION

From the information on the data, there are a number of columns in the dataset that are irrelevant. There are a number of columns dealing with post-crash details such as injuries as well as other administrative details,there are also columns with a high number of missing values.

#### People Dataset

In [7]:
people.head()

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,SEX,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,M,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,M,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,X,...,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,NaN,X,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,NaN,X,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [8]:
people.shape

(1932026, 29)

In [9]:
people.columns

Index(['PERSON_ID', 'PERSON_TYPE', 'CRASH_RECORD_ID', 'VEHICLE_ID',
       'CRASH_DATE', 'SEAT_NO', 'CITY', 'STATE', 'ZIPCODE', 'SEX', 'AGE',
       'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'SAFETY_EQUIPMENT',
       'AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'HOSPITAL',
       'EMS_AGENCY', 'EMS_RUN_NO', 'DRIVER_ACTION', 'DRIVER_VISION',
       'PHYSICAL_CONDITION', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY',
       'PEDPEDAL_LOCATION', 'BAC_RESULT', 'BAC_RESULT VALUE',
       'CELL_PHONE_USE'],
      dtype='object')

In [10]:
percentage_null(people)

PERSON_ID                 0.000000
PERSON_TYPE               0.000000
CRASH_RECORD_ID           0.000000
VEHICLE_ID                2.040811
CRASH_DATE                0.000000
SEAT_NO                  79.765645
CITY                     27.243112
STATE                    26.138209
ZIPCODE                  33.104834
SEX                       1.682172
AGE                      29.147382
DRIVERS_LICENSE_STATE    41.469680
DRIVERS_LICENSE_CLASS    51.276277
SAFETY_EQUIPMENT          0.278102
AIRBAG_DEPLOYED           1.963328
EJECTION                  1.248275
INJURY_CLASSIFICATION     0.038457
HOSPITAL                 83.705551
EMS_AGENCY               89.933572
EMS_RUN_NO               98.319433
DRIVER_ACTION            20.413597
DRIVER_VISION            20.444135
PHYSICAL_CONDITION       20.359095
PEDPEDAL_ACTION          98.051734
PEDPEDAL_VISIBILITY      98.055254
PEDPEDAL_LOCATION        98.051734
BAC_RESULT               20.364685
BAC_RESULT VALUE         99.890529
CELL_PHONE_USE      

In [11]:
# Dropping the unnecessary columns
people.drop(columns=['VEHICLE_ID','CITY', 'STATE', 'ZIPCODE','DRIVERS_LICENSE_STATE', 'SEAT_NO','HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO',
                     'PEDPEDAL_ACTION','PEDPEDAL_VISIBILITY','PEDPEDAL_LOCATION','BAC_RESULT' ,'BAC_RESULT VALUE'],inplace=True)

In [12]:
people['SEX'].value_counts()

SEX
M    998706
F    725372
X    175448
Name: count, dtype: int64

In [13]:
people['INJURY_CLASSIFICATION'].value_counts()

INJURY_CLASSIFICATION
NO INDICATION OF INJURY     1760608
NONINCAPACITATING INJURY      95768
REPORTED, NOT EVIDENT         56296
INCAPACITATING INJURY         17546
FATAL                          1065
Name: count, dtype: int64

In [14]:
people['AIRBAG_DEPLOYED'].value_counts()

AIRBAG_DEPLOYED
DID NOT DEPLOY                            970169
NOT APPLICABLE                            408560
DEPLOYMENT UNKNOWN                        387375
DEPLOYED, FRONT                            60276
DEPLOYED, COMBINATION                      49257
DEPLOYED, SIDE                             17510
DEPLOYED OTHER (KNEE, AIR, BELT, ETC.)       947
Name: count, dtype: int64

In [15]:
airbag_dict = {'DID NOT DEPLOY': 'DID NOT DEPLOY',
                'NOT APPLICABLE': 'NOT APPLICABLE/UNKNOWN',
                'DEPLOYMENT UNKNOWN': 'NOT APPLICABLE/UNKNOWN',
                'DEPLOYED, FRONT': 'DEPLOYED',
                'DEPLOYED, COMBINATION': 'DEPLOYED',
                'DEPLOYED, SIDE': 'DEPLOYED',
                'DEPLOYED OTHER (KNEE, AIR, BELT, ETC.)': 'DEPLOYED'}

people.AIRBAG_DEPLOYED = people.AIRBAG_DEPLOYED.map(airbag_dict)
people.AIRBAG_DEPLOYED.value_counts()

AIRBAG_DEPLOYED
DID NOT DEPLOY            970169
NOT APPLICABLE/UNKNOWN    795935
DEPLOYED                  127990
Name: count, dtype: int64

In [16]:
people['EJECTION'].value_counts()

EJECTION
NONE                  1778140
UNKNOWN                121381
TOTALLY EJECTED          5792
PARTIALLY EJECTED        1428
TRAPPED/EXTRICATED       1168
Name: count, dtype: int64

In [17]:
people['SAFETY_EQUIPMENT'].value_counts()

SAFETY_EQUIPMENT
USAGE UNKNOWN                                 914551
SAFETY BELT USED                              899285
NONE PRESENT                                   65132
SAFETY BELT NOT USED                           10516
HELMET NOT USED                                 9709
CHILD RESTRAINT USED                            7822
CHILD RESTRAINT - FORWARD FACING                4945
BICYCLE HELMET (PEDACYCLIST INVOLVED ONLY)      3806
CHILD RESTRAINT - TYPE UNKNOWN                  2599
CHILD RESTRAINT - REAR FACING                   2250
DOT COMPLIANT MOTORCYCLE HELMET                 1527
HELMET USED                                     1358
BOOSTER SEAT                                    1285
CHILD RESTRAINT NOT USED                         853
NOT DOT COMPLIANT MOTORCYCLE HELMET              297
SHOULD/LAP BELT USED IMPROPERLY                  270
WHEELCHAIR                                       237
CHILD RESTRAINT USED IMPROPERLY                  165
STRETCHER                    

In [18]:
safety_dict = {'USAGE UNKNOWN': 'USAGE UNKNOWN',
                'SAFETY BELT USED': 'SAFETY EQUIPMENT USED',
                'NONE PRESENT': 'NONE PRESENT/USED',
                'BICYCLE HELMET (PEDACYCLIST INVOLVED ONLY)': 'SAFETY EQUIPMENT USED',
                'HELMET NOT USED': 'NONE PRESENT/USED',
                'SAFETY BELT NOT USED': 'NONE PRESENT/USED',
                'DOT COMPLIANT MOTORCYCLE HELMET ': 'SAFETY EQUIPMENT USED',
                'CHILD RESTRAINT - FORWARD FACING': 'SAFETY EQUIPMENT USED',
                'CHILD RESTRAINT - REAR FACING': 'SAFETY EQUIPMENT USED',
                'CHILD RESTRAINT - TYPE UNKNOWN': 'SAFETY EQUIPMENT USED',
                'SHOULD/LAP BELT USED IMPROPERLY': 'NONE PRESENT/USED'}

people.SAFETY_EQUIPMENT = people.SAFETY_EQUIPMENT.map(safety_dict)
people.SAFETY_EQUIPMENT.value_counts()

SAFETY_EQUIPMENT
USAGE UNKNOWN            914551
SAFETY EQUIPMENT USED    912885
NONE PRESENT/USED         85627
Name: count, dtype: int64

In [19]:
people['DRIVER_VISION'].value_counts()

DRIVER_VISION
NOT OBSCURED              768702
UNKNOWN                   731742
OTHER                      15099
MOVING VEHICLES             8633
PARKED VEHICLES             5300
WINDSHIELD (WATER/ICE)      4093
BLINDED - SUNLIGHT          1833
TREES, PLANTS                610
BUILDINGS                    540
BLINDED - HEADLIGHTS         164
BLOWING MATERIALS            105
HILLCREST                    100
EMBANKMENT                    83
SIGNBOARD                     36
Name: count, dtype: int64

In [20]:
driver_dict = {'NOT OBSCURED': 'NOT OBSCURED',
                'UNKNOWN': 'UNKNOWN',
                'OTHER': 'OBSCURED',
                'PARKED VEHICLES': 'OBSCURED',
                'MOVING VEHICLES': 'OBSCURED',
                'BLINDED - SUNLIGHT': 'OBSCURED'}

people.DRIVER_VISION = people.DRIVER_VISION.map(driver_dict)
people.DRIVER_VISION.value_counts()

DRIVER_VISION
NOT OBSCURED    768702
UNKNOWN         731742
OBSCURED         30865
Name: count, dtype: int64

In [21]:
people['DRIVER_ACTION'].value_counts()

DRIVER_ACTION
NONE                                 547534
UNKNOWN                              390413
FAILED TO YIELD                      139815
OTHER                                138288
FOLLOWED TOO CLOSELY                  90171
IMPROPER BACKING                      45292
IMPROPER TURN                         40450
IMPROPER LANE CHANGE                  39526
IMPROPER PASSING                      34504
DISREGARDED CONTROL DEVICES           27211
TOO FAST FOR CONDITIONS               22593
WRONG WAY/SIDE                         6209
IMPROPER PARKING                       5651
OVERCORRECTED                          3033
EVADING POLICE VEHICLE                 2408
CELL PHONE USE OTHER THAN TEXTING      2243
EMERGENCY VEHICLE ON CALL              1436
TEXTING                                 604
STOPPED SCHOOL BUS                      183
LICENSE RESTRICTIONS                     66
Name: count, dtype: int64

In [22]:
people['PHYSICAL_CONDITION'].value_counts()

PHYSICAL_CONDITION
NORMAL                          997117
UNKNOWN                         513151
IMPAIRED - ALCOHOL                6480
REMOVED BY EMS                    5588
OTHER                             4479
EMOTIONAL                         4044
FATIGUED/ASLEEP                   4027
ILLNESS/FAINTED                   1380
HAD BEEN DRINKING                 1108
IMPAIRED - DRUGS                   720
IMPAIRED - ALCOHOL AND DRUGS       400
MEDICATED                          189
Name: count, dtype: int64

In [23]:
people['INJURY_CLASSIFICATION'].value_counts()

INJURY_CLASSIFICATION
NO INDICATION OF INJURY     1760608
NONINCAPACITATING INJURY      95768
REPORTED, NOT EVIDENT         56296
INCAPACITATING INJURY         17546
FATAL                          1065
Name: count, dtype: int64

In [24]:
people['AGE'].value_counts()

AGE
 25.0     38298
 26.0     38295
 27.0     38269
 28.0     37686
 24.0     37107
          ...  
-47.0         1
-49.0         1
-177.0        1
-40.0         1
-59.0         1
Name: count, Length: 117, dtype: int64

In [25]:
#select the useful features
features_p = people[['CRASH_RECORD_ID','AGE','INJURY_CLASSIFICATION','PERSON_ID','SEX','PERSON_TYPE','PHYSICAL_CONDITION']]

In [26]:
people_df = features_p.copy()

In [27]:
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1932026 entries, 0 to 1932025
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   CRASH_RECORD_ID        object 
 1   AGE                    float64
 2   INJURY_CLASSIFICATION  object 
 3   PERSON_ID              object 
 4   SEX                    object 
 5   PERSON_TYPE            object 
 6   PHYSICAL_CONDITION     object 
dtypes: float64(1), object(6)
memory usage: 103.2+ MB


#### Vehicles Dataset

In [28]:
vehicles.head()

,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,MODEL,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,1727162,f5943b05f46b8d4148a63b7506a59113eae0cf1075aabc...,12/21/2023 08:57:00 AM,2,PEDESTRIAN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1717556,7b1763088507f77e0e552c009a6bf89a4d6330c7527706...,12/06/2023 03:24:00 PM,1,DRIVER,NaN,1634931.0,NaN,NISSAN,SENTRA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1717574,2603ff5a88f0b9b54576934c5ed4e4a64e8278e005687b...,12/06/2023 04:00:00 PM,2,DRIVER,NaN,1634978.0,NaN,CHRYSLER,SEBRING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1717579,a52ef70e33d468b855b5be44e8638a564434dcf99c0edf...,12/06/2023 04:30:00 PM,1,DRIVER,NaN,1634948.0,NaN,SUBARU,OUTBACK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1720118,609055f4b1a72a44d6ec40ba9036cefd7c1287a755eb6c...,12/10/2023 12:12:00 PM,1,DRIVER,NaN,1637401.0,NaN,TOYOTA,RAV4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
vehicles.shape

(1794597, 71)

In [30]:
vehicles.columns

Index(['CRASH_UNIT_ID', 'CRASH_RECORD_ID', 'CRASH_DATE', 'UNIT_NO',
       'UNIT_TYPE', 'NUM_PASSENGERS', 'VEHICLE_ID', 'CMRC_VEH_I', 'MAKE',
       'MODEL', 'LIC_PLATE_STATE', 'VEHICLE_YEAR', 'VEHICLE_DEFECT',
       'VEHICLE_TYPE', 'VEHICLE_USE', 'TRAVEL_DIRECTION', 'MANEUVER',
       'TOWED_I', 'FIRE_I', 'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I', 'TOWED_BY',
       'TOWED_TO', 'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I',
       'AREA_04_I', 'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I',
       'AREA_09_I', 'AREA_10_I', 'AREA_11_I', 'AREA_12_I', 'AREA_99_I',
       'FIRST_CONTACT_POINT', 'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO',
       'COMMERCIAL_SRC', 'GVWR', 'CARRIER_NAME', 'CARRIER_STATE',
       'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO',
       'HAZMAT_PRESENT_I', 'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO',
       'MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I',
       'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO', 'WIDE_LOAD_I',
       'TRAILER1_

In [31]:
percentage_null(vehicles)

CRASH_UNIT_ID               0.000000
CRASH_RECORD_ID             0.000000
CRASH_DATE                  0.000000
UNIT_NO                     0.000000
UNIT_TYPE                   0.123928
                             ...    
CARGO_BODY_TYPE            99.200823
LOAD_TYPE                  99.236096
HAZMAT_OUT_OF_SERVICE_I    99.301737
MCS_OUT_OF_SERVICE_I       99.288643
HAZMAT_CLASS               99.935919
Length: 71, dtype: float64

In [32]:
vehicles.drop(columns=['CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO','COMMERCIAL_SRC', 'GVWR', 'CARRIER_NAME', 'CARRIER_STATE',      'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO',
       'HAZMAT_PRESENT_I', 'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO','MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I',
       'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO', 'WIDE_LOAD_I','TRAILER1_WIDTH', 'TRAILER2_WIDTH', 'TRAILER1_LENGTH',
       'TRAILER2_LENGTH', 'TOTAL_VEHICLE_LENGTH', 'AXLE_CNT', 'VEHICLE_CONFIG','CARGO_BODY_TYPE', 'LOAD_TYPE', 
       'HAZMAT_OUT_OF_SERVICE_I','MCS_OUT_OF_SERVICE_I', 'HAZMAT_CLASS','EXCEED_SPEED_LIMIT_I', 'TOWED_BY','TOWED_TO','LIC_PLATE_STATE', 'VEHICLE_YEAR',
       'TOWED_I', 'FIRE_I','CMRC_VEH_I','NUM_PASSENGERS', 'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I', 'AREA_04_I', 'AREA_05_I', 'AREA_06_I', 'AREA_07_I',
        'AREA_08_I', 'AREA_09_I', 'AREA_10_I', 'AREA_11_I', 'AREA_12_I', 'AREA_99_I'], inplace=True)

In [33]:
vehicles['FIRST_CONTACT_POINT'].value_counts()

FIRST_CONTACT_POINT
FRONT                 343789
REAR                  231582
UNKNOWN               167200
SIDE-LEFT             120614
FRONT-LEFT-CORNER     115638
SIDE-RIGHT            114756
FRONT-RIGHT-CORNER    113289
FRONT-LEFT             81524
FRONT-RIGHT            76903
REAR-LEFT              68088
REAR-LEFT-CORNER       51448
OTHER                  41611
REAR-RIGHT-CORNER      37611
REAR-RIGHT             36488
SIDE-LEFT-REAR         29380
TOTAL (ALL AREAS)      28015
SIDE-RIGHT-REAR        22252
SIDE-LEFT-FRONT        19148
SIDE-RIGHT-FRONT       16534
NONE                   13790
ROOF                   11958
UNDER CARRIAGE          5844
TOP                     2188
Name: count, dtype: int64

In [34]:
vehicles['MAKE'].value_counts()

MAKE
CHEVROLET                                          202430
FORD                                               175097
UNKNOWN                                            174476
NISSAN                                             141370
HONDA                                              129165
                                                    ...  
KEMPF CAR HAULER                                        1
SNOWBIRD TRAILER COMPANY, INC. (IRON RIDGE, WI)         1
LONG TRAILER COMPANY, INC.                              1
FELBURN FLATBED TRAILER                                 1
ACADEMY MOBILE HOME CORPORATION                         1
Name: count, Length: 1380, dtype: int64

In [35]:
vehicles['MODEL'].value_counts()

MODEL
UNKNOWN                                  180803
OTHER (EXPLAIN IN NARRATIVE)             167044
CAMRY                                     55820
COROLLA                                   35347
CIVIC                                     33620
                                          ...  
KEMPF CAR HAULER                              1
SAVAGE COACH MFG., INC.                       1
KENT INDUSTRIES INC., BRISTOL, IN             1
164                                           1
HOGER (MFD. BY SUNDOWNER TRAILER CO.)         1
Name: count, Length: 2769, dtype: int64

In [36]:
vehicles['UNIT_TYPE'].value_counts()

UNIT_TYPE
DRIVER                 1502188
PARKED                  236646
PEDESTRIAN               22700
DRIVERLESS               14353
BICYCLE                  14263
NON-MOTOR VEHICLE         1618
NON-CONTACT VEHICLE        324
DISABLED VEHICLE           274
EQUESTRIAN                   7
Name: count, dtype: int64

In [37]:
vehicles['MANEUVER'].value_counts()

MANEUVER
STRAIGHT AHEAD                        811005
PARKED                                240846
UNKNOWN/NA                            133396
SLOW/STOP IN TRAFFIC                  128162
TURNING LEFT                          104575
BACKING                                70611
TURNING RIGHT                          58471
PASSING/OVERTAKING                     42694
CHANGING LANES                         33529
OTHER                                  29485
ENTERING TRAFFIC LANE FROM PARKING     20630
MERGING                                12314
U-TURN                                 10047
STARTING IN TRAFFIC                     9822
LEAVING TRAFFIC LANE TO PARK            8441
AVOIDING VEHICLES/OBJECTS               7378
SKIDDING/CONTROL LOSS                   6483
ENTER FROM DRIVE/ALLEY                  6207
PARKED IN TRAFFIC LANE                  5420
SLOW/STOP - LEFT TURN                   3042
DRIVING WRONG WAY                       2634
NEGOTIATING A CURVE                     2136
S

In [38]:
vehicles['VEHICLE_TYPE'].value_counts()

VEHICLE_TYPE
PASSENGER                                 1100811
SPORT UTILITY VEHICLE (SUV)                242858
UNKNOWN/NA                                 160413
VAN/MINI-VAN                                82399
PICKUP                                      57260
TRUCK - SINGLE UNIT                         32895
OTHER                                       21477
BUS OVER 15 PASS.                           19085
TRACTOR W/ SEMI-TRAILER                     16563
BUS UP TO 15 PASS.                           5124
MOTORCYCLE (OVER 150CC)                      4291
SINGLE UNIT TRUCK WITH TRAILER               2962
OTHER VEHICLE WITH TRAILER                   2417
TRACTOR W/O SEMI-TRAILER                     2184
AUTOCYCLE                                     678
MOPED OR MOTORIZED BICYCLE                    654
MOTOR DRIVEN CYCLE                            328
ALL-TERRAIN VEHICLE (ATV)                     199
FARM EQUIPMENT                                 87
3-WHEELED MOTORCYCLE (2 REAR WHEELS) 

In [39]:
vehicles['VEHICLE_DEFECT'].value_counts()

VEHICLE_DEFECT
NONE                938798
UNKNOWN             795030
OTHER                10275
BRAKES                5410
TIRES                  884
STEERING               797
WHEELS                 436
SUSPENSION             279
FUEL SYSTEM            260
ENGINE/MOTOR           223
WINDOWS                115
LIGHTS                 108
CARGO                   64
SIGNALS                 42
RESTRAINT SYSTEM        27
TRAILER COUPLING        24
EXHAUST                 21
Name: count, dtype: int64

In [40]:
vehicles['VEHICLE_USE'].value_counts()

VEHICLE_USE
PERSONAL                        1132462
UNKNOWN/NA                       358091
NOT IN USE                        90373
OTHER                             55396
TAXI/FOR HIRE                     21239
COMMERCIAL - SINGLE UNIT          21030
RIDESHARE SERVICE                 14463
CTA                               12620
POLICE                            11533
CONSTRUCTION/MAINTENANCE           7789
COMMERCIAL - MULTI-UNIT            7003
SCHOOL BUS                         4821
OTHER TRANSIT                      3964
TOW TRUCK                          3378
AMBULANCE                          2119
FIRE                               1790
STATE OWNED                        1510
DRIVER EDUCATION                   1336
MASS TRANSIT                        832
LAWN CARE/LANDSCAPING               627
AGRICULTURE                         189
CAMPER/RV - SINGLE UNIT             111
MILITARY                             68
HOUSE TRAILER                        30
CAMPER/RV - TOWED/MULTI-UNIT

In [41]:
vehicles['FIRST_CONTACT_POINT'].value_counts()

FIRST_CONTACT_POINT
FRONT                 343789
REAR                  231582
UNKNOWN               167200
SIDE-LEFT             120614
FRONT-LEFT-CORNER     115638
SIDE-RIGHT            114756
FRONT-RIGHT-CORNER    113289
FRONT-LEFT             81524
FRONT-RIGHT            76903
REAR-LEFT              68088
REAR-LEFT-CORNER       51448
OTHER                  41611
REAR-RIGHT-CORNER      37611
REAR-RIGHT             36488
SIDE-LEFT-REAR         29380
TOTAL (ALL AREAS)      28015
SIDE-RIGHT-REAR        22252
SIDE-LEFT-FRONT        19148
SIDE-RIGHT-FRONT       16534
NONE                   13790
ROOF                   11958
UNDER CARRIAGE          5844
TOP                     2188
Name: count, dtype: int64

In [42]:
# select the useful features
features_v = vehicles[['CRASH_RECORD_ID', 'VEHICLE_TYPE','VEHICLE_USE','VEHICLE_DEFECT','MANEUVER','UNIT_TYPE']]

In [43]:
vehicles_df = features_v.copy()

#### Crashes Dataset

In [44]:
crash.shape

(879740, 48)

In [45]:
crash[crash['CRASH_DATE_EST_I'].notnull()]

,CRASH_RECORD_ID,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
8,36360857c079418cba1b1d70cf653595bbfb4566de8fcb...,Y,01/01/2022 04:32:00 PM,10,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",ANGLE,PARKING LOT,...,0.0,0.0,2.0,0.0,16,7,1,NaN,NaN,NaN
63,8640aa03a86f10df7a8c1dafc03d1551c4a4c0262397f3...,Y,09/06/2023 11:00:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0,0.0,3.0,0.0,11,4,9,41.937252,-87.776321,POINT (-87.776320528037 41.937252170984)
86,3b3558c02e1b695ff6f79d6b4bcaf8cb220eb83d519454...,Y,08/11/2023 10:15:00 PM,5,NO CONTROLS,NO CONTROLS,UNKNOWN,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,PARKING LOT,...,0.0,0.0,1.0,0.0,22,6,8,NaN,NaN,NaN
94,bde92bfca22985fc0b0354b01d1eb8b71e312ff6b171c1...,Y,08/20/2023 07:00:00 AM,30,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,PARKED MOTOR VEHICLE,DRIVEWAY,...,0.0,0.0,1.0,0.0,7,1,8,41.777122,-87.692925,POINT (-87.692925138017 41.777121674979)
101,d933cc5ab49d3b7a1d1429fdcf1611a3398d13dbbde9aa...,Y,09/21/2023 08:30:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,FOUR WAY,...,0.0,0.0,2.0,0.0,20,5,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879647,e860739bf6371379aac2780cea1f1db2963a7b7fa6c5bd...,Y,10/03/2024 07:00:00 PM,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DUSK,FIXED OBJECT,ROUNDABOUT,...,0.0,0.0,1.0,0.0,19,5,10,41.740685,-87.568559,POINT (-87.568559315121 41.74068525818)
879670,d996490d497d4a688a68a84e64d3b8e4ca8b5287d3234f...,N,10/03/2024 09:00:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,OTHER OBJECT,OTHER,...,0.0,0.0,1.0,0.0,9,5,10,41.704657,-87.656968,POINT (-87.656967709864 41.704657080424)
879688,9bed452ceab86a1274675a7f9c3ac17ca19643c72b2e7c...,Y,08/28/2024 01:00:00 PM,20,NO CONTROLS,NO CONTROLS,UNKNOWN,UNKNOWN,PARKED MOTOR VEHICLE,ONE-WAY,...,0.0,0.0,1.0,0.0,13,4,8,41.888296,-87.753988,POINT (-87.753988262533 41.888295726666)
879731,5914c45ccb3f73c5ad90016bbbc05e4d6ed245dc2ba4e5...,Y,09/27/2019 04:25:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,PEDESTRIAN,NOT DIVIDED,...,1.0,0.0,1.0,0.0,16,6,9,41.736402,-87.613906,POINT (-87.613906262901 41.736402347964)


In [46]:
admin_data = ['CRASH_DATE_EST_I','REPORT_TYPE','DATE_POLICE_NOTIFIED','PHOTOS_TAKEN_I','STATEMENTS_TAKEN_I']
data_no_admin = crash.drop(columns = admin_data,axis = 1)

In [47]:
data_no_admin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879740 entries, 0 to 879739
Data columns (total 43 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                879740 non-null  object 
 1   CRASH_DATE                     879740 non-null  object 
 2   POSTED_SPEED_LIMIT             879740 non-null  int64  
 3   TRAFFIC_CONTROL_DEVICE         879740 non-null  object 
 4   DEVICE_CONDITION               879740 non-null  object 
 5   WEATHER_CONDITION              879740 non-null  object 
 6   LIGHTING_CONDITION             879740 non-null  object 
 7   FIRST_CRASH_TYPE               879740 non-null  object 
 8   TRAFFICWAY_TYPE                879740 non-null  object 
 9   LANE_CNT                       199017 non-null  float64
 10  ALIGNMENT                      879740 non-null  object 
 11  ROADWAY_SURFACE_COND           879740 non-null  object 
 12  ROAD_DEFECT                   

In [48]:
data_no_admin[data_no_admin['LANE_CNT'].notnull()]

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
17,4502c89bb17d2a4817c517541350b1470636cc9c29e6fd...,02/19/2019 05:30:00 AM,20,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,REAR END,ONE-WAY,2.0,...,0.0,0.0,2.0,0.0,5,3,2,NaN,NaN,NaN
81,87a5f1913061fbfc022966ccc7949e7b93507cee6cb27e...,11/02/2016 04:49:00 PM,25,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DUSK,ANGLE,OTHER,4.0,...,0.0,0.0,2.0,0.0,16,4,11,NaN,NaN,NaN
131,b8bede6e04aeda61e7fd297157a58feff4cc86a0cabcc6...,08/26/2018 12:11:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,OTHER,4.0,...,0.0,0.0,2.0,0.0,12,1,8,NaN,NaN,NaN
177,dcfd1fbeacb8a918e7404fdb0dd8e9faadcbc6c8dececf...,06/22/2018 05:27:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,FIXED OBJECT,OTHER,0.0,...,0.0,0.0,0.0,0.0,17,6,6,NaN,NaN,NaN
546,01fc5f2a9657da8969b3f0c3eaab0b20da2a81ecac5023...,12/16/2018 10:45:00 AM,15,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,ONE-WAY,4.0,...,0.0,0.0,3.0,0.0,10,1,12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879362,faccdde92fb871184f68228bce995062012d66b89b1b38...,03/27/2019 09:36:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,NOT DIVIDED,4.0,...,0.0,0.0,2.0,0.0,21,4,3,41.880834,-87.725797,POINT (-87.725796597174 41.880834299772)
879372,fc637240524f494b339c8417c0527bd011ed4a08bf7077...,05/24/2018 08:30:00 PM,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DUSK,TURNING,DIVIDED - W/MEDIAN (NOT RAISED),2.0,...,0.0,0.0,2.0,0.0,20,5,5,41.726851,-87.648788,POINT (-87.648787911154 41.726850549145)
879380,fdd84df84cadb7e639b177a06068f95a6f35e367aba8eb...,03/03/2018 07:11:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,2.0,...,0.0,3.0,3.0,0.0,7,7,3,41.765291,-87.644562,POINT (-87.644562127496 41.765291024572)
879713,67dd84a904e46800b722c9f368de8b3e634864ff30099a...,03/04/2019 06:40:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,4.0,...,0.0,0.0,2.0,0.0,18,2,3,41.953760,-87.727480,POINT (-87.727480334329 41.953759575772)


In [49]:
data_no_admin = data_no_admin.drop(['LANE_CNT'], axis = 1)
data_no_admin.head()

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,09/05/2023 07:05:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DUSK,ANGLE,"FIVE POINT, OR MORE",STRAIGHT AND LEVEL,...,2.0,0.0,2.0,0.0,19,3,9,NaN,NaN,NaN
1,2675c13fd0f474d730a5b780968b3cafc7c12d7adb661f...,09/22/2023 06:45:00 PM,50,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,18,6,9,NaN,NaN,NaN
2,5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4...,07/29/2023 02:45:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,0.0,0.0,1.0,0.0,14,7,7,41.85412,-87.665902,POINT (-87.665902342962 41.854120262952)
3,7ebf015016f83d09b321afd671a836d6b148330535d5df...,08/09/2023 11:00:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,23,4,8,NaN,NaN,NaN
4,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,08/18/2023 12:50:00 PM,15,OTHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,OTHER,STRAIGHT AND LEVEL,...,1.0,0.0,1.0,0.0,12,6,8,NaN,NaN,NaN


In [50]:
data_no_admin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879740 entries, 0 to 879739
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                879740 non-null  object 
 1   CRASH_DATE                     879740 non-null  object 
 2   POSTED_SPEED_LIMIT             879740 non-null  int64  
 3   TRAFFIC_CONTROL_DEVICE         879740 non-null  object 
 4   DEVICE_CONDITION               879740 non-null  object 
 5   WEATHER_CONDITION              879740 non-null  object 
 6   LIGHTING_CONDITION             879740 non-null  object 
 7   FIRST_CRASH_TYPE               879740 non-null  object 
 8   TRAFFICWAY_TYPE                879740 non-null  object 
 9   ALIGNMENT                      879740 non-null  object 
 10  ROADWAY_SURFACE_COND           879740 non-null  object 
 11  ROAD_DEFECT                    879740 non-null  object 
 12  CRASH_TYPE                    

In [51]:
data_no_admin[data_no_admin['INTERSECTION_RELATED_I'].notnull()]

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,09/05/2023 07:05:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DUSK,ANGLE,"FIVE POINT, OR MORE",STRAIGHT AND LEVEL,...,2.0,0.0,2.0,0.0,19,3,9,NaN,NaN,NaN
10,b236c77d59e32b7b469a6e2f17f438b7457e1bd8bc689b...,07/29/2023 01:00:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,0.0,0.0,3.0,0.0,13,7,7,NaN,NaN,NaN
11,3b6d23138e3f009e54ae5ef25061e9e96fe44fd0e62afa...,12/09/2021 10:30:00 AM,25,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,...,0.0,0.0,3.0,0.0,10,5,12,NaN,NaN,NaN
14,14386daec6219c6032b71612b28f0e4cd38e2898f39aae...,08/13/2023 12:11:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,FOUR WAY,STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,0,1,8,NaN,NaN,NaN
20,c9053169f9e6d24f3035623ecfd0a70c1cb3adba409b7d...,09/22/2023 10:46:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,0.0,0.0,0.0,0.0,22,6,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879714,bf86786457a08ebccd20d6480ed2c5c295e04a51004578...,06/09/2020 08:00:00 PM,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,RAIN,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,20,3,6,41.777927,-87.638835,POINT (-87.638834655301 41.777926777918)
879717,bfa8377168b8dc75b4a09bf627b4e28fc77f465911c99b...,09/13/2023 01:08:00 PM,30,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,TURNING,FOUR WAY,STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,13,4,9,41.923913,-87.637409,POINT (-87.637408804478 41.923913444256)
879719,5bda1ba236fc7e7206dd5ed31f57591dbe50ff8134e707...,12/16/2022 12:10:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,FOUR WAY,STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,12,6,12,41.916235,-87.667838,POINT (-87.66783777213 41.916235337469)
879729,bc2876dcd7c4098806301cb646232eb8f65c86a4f418b7...,05/28/2023 01:20:00 AM,30,TRAFFIC SIGNAL,UNKNOWN,UNKNOWN,UNKNOWN,REAR END,FOUR WAY,STRAIGHT AND LEVEL,...,0.0,0.0,3.0,0.0,1,1,5,41.851497,-87.719932,POINT (-87.719931713533 41.851497025522)


In [52]:
y_cols = ['INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I','HIT_AND_RUN_I', 'DOORING_I', 'WORK_ZONE_I', 'WORKERS_PRESENT_I']
for col in y_cols:
    print(data_no_admin[col].value_counts())
    print('Number of nulls:' , data_no_admin[col].isnull().sum())
    print("")

INTERSECTION_RELATED_I
Y    192163
N      9619
Name: count, dtype: int64
Number of nulls: 677958

NOT_RIGHT_OF_WAY_I
Y    36507
N     3689
Name: count, dtype: int64
Number of nulls: 839544

HIT_AND_RUN_I
Y    263997
N     11839
Name: count, dtype: int64
Number of nulls: 603904

DOORING_I
Y    1874
N     909
Name: count, dtype: int64
Number of nulls: 876957

WORK_ZONE_I
Y    3830
N    1130
Name: count, dtype: int64
Number of nulls: 874780

WORKERS_PRESENT_I
Y    1128
N     147
Name: count, dtype: int64
Number of nulls: 878465



In [53]:
# Replace the Ys with 1 and every other entry with 0
for col in y_cols:
    data_no_admin[col] = data_no_admin[col].map(lambda x: 1 if x =='Y' else 0)
    print(data_no_admin[col].value_counts())
    print('Number of nulls:' , data_no_admin[col].isnull().sum())
    print("")

INTERSECTION_RELATED_I
0    687577
1    192163
Name: count, dtype: int64
Number of nulls: 0

NOT_RIGHT_OF_WAY_I
0    843233
1     36507
Name: count, dtype: int64
Number of nulls: 0

HIT_AND_RUN_I
0    615743
1    263997
Name: count, dtype: int64
Number of nulls: 0

DOORING_I
0    877866
1      1874
Name: count, dtype: int64
Number of nulls: 0

WORK_ZONE_I
0    875910
1      3830
Name: count, dtype: int64
Number of nulls: 0

WORKERS_PRESENT_I
0    878612
1      1128
Name: count, dtype: int64
Number of nulls: 0



In [54]:
data_no_admin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879740 entries, 0 to 879739
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                879740 non-null  object 
 1   CRASH_DATE                     879740 non-null  object 
 2   POSTED_SPEED_LIMIT             879740 non-null  int64  
 3   TRAFFIC_CONTROL_DEVICE         879740 non-null  object 
 4   DEVICE_CONDITION               879740 non-null  object 
 5   WEATHER_CONDITION              879740 non-null  object 
 6   LIGHTING_CONDITION             879740 non-null  object 
 7   FIRST_CRASH_TYPE               879740 non-null  object 
 8   TRAFFICWAY_TYPE                879740 non-null  object 
 9   ALIGNMENT                      879740 non-null  object 
 10  ROADWAY_SURFACE_COND           879740 non-null  object 
 11  ROAD_DEFECT                    879740 non-null  object 
 12  CRASH_TYPE                    

In [55]:
data_no_admin[data_no_admin['WORK_ZONE_TYPE'].notnull()]

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
216,555ed321400f9ef0f6a74300f63ab41044b5a22aee83d3...,09/19/2023 03:20:00 PM,45,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,OTHER,STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,15,3,9,NaN,NaN,NaN
475,25320f6cbb540dde38a4af7cf8111089c214fc22be1d89...,09/18/2023 09:00:00 AM,30,OTHER WARNING SIGN,UNKNOWN,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,9,2,9,41.979255,-87.728189,POINT (-87.728188507001 41.979254802019)
769,0474a8dfe651dd4d4a027063236d011a931bc2936551c6...,09/21/2018 06:27:00 AM,35,OTHER,FUNCTIONING PROPERLY,CLOUDY/OVERCAST,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN BARRIER,"CURVE, LEVEL",...,0.0,0.0,4.0,0.0,6,6,9,NaN,NaN,NaN
775,05995ddabf59b34e77521980ffe2735b6560c8358df8b8...,05/17/2022 02:45:00 PM,15,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,OTHER,STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,14,3,5,41.916960,-87.701927,POINT (-87.701927401978 41.91696035001)
950,0b20ea43ef78272a119398871f0bbf10be79772c30ad8a...,07/24/2023 09:00:00 PM,40,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,ONE-WAY,STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,21,2,7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877992,1a5a642e5cb0eed349719ef60f7b9be58921344da89347...,05/23/2023 09:20:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,OTHER OBJECT,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,9,3,5,41.872041,-87.633645,POINT (-87.633645484534 41.872040758511)
878690,8f1cce66e7bc8648b352e60a71cd6aefcdcfa6fc4f7318...,08/14/2019 10:50:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,0.0,1.0,2.0,0.0,10,4,8,41.891647,-87.637590,POINT (-87.637590143245 41.891647060358)
878746,97ce4aa573a05f9ff4416303a396889d015d1e3c4ea884...,04/22/2023 10:30:00 PM,30,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,FIXED OBJECT,UNKNOWN,STRAIGHT AND LEVEL,...,0.0,0.0,1.0,0.0,22,7,4,41.963266,-87.646922,POINT (-87.646922415275 41.963266406281)
879276,ecbe2d0c3524c7c8c97eed7ae6ee7d01e5055f22ef5fe4...,06/28/2023 10:05:00 AM,25,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,PEDESTRIAN,ONE-WAY,STRAIGHT AND LEVEL,...,0.0,0.0,1.0,0.0,10,4,6,41.874331,-87.639601,POINT (-87.639600735438 41.874330690014)


In [56]:
data_no_admin['WORK_ZONE_TYPE'].value_counts()

WORK_ZONE_TYPE
CONSTRUCTION    2654
UNKNOWN          542
MAINTENANCE      393
UTILITY          241
Name: count, dtype: int64

In [57]:
#Dropping work zone type because it is not related to other entries in the dataset
data_no_admin = data_no_admin.drop(['WORK_ZONE_TYPE'], axis = 1)
data_no_admin.head()

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,09/05/2023 07:05:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DUSK,ANGLE,"FIVE POINT, OR MORE",STRAIGHT AND LEVEL,...,2.0,0.0,2.0,0.0,19,3,9,NaN,NaN,NaN
1,2675c13fd0f474d730a5b780968b3cafc7c12d7adb661f...,09/22/2023 06:45:00 PM,50,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,18,6,9,NaN,NaN,NaN
2,5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4...,07/29/2023 02:45:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,0.0,0.0,1.0,0.0,14,7,7,41.85412,-87.665902,POINT (-87.665902342962 41.854120262952)
3,7ebf015016f83d09b321afd671a836d6b148330535d5df...,08/09/2023 11:00:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,23,4,8,NaN,NaN,NaN
4,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,08/18/2023 12:50:00 PM,15,OTHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,OTHER,STRAIGHT AND LEVEL,...,1.0,0.0,1.0,0.0,12,6,8,NaN,NaN,NaN


In [58]:
data_no_admin['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
UNABLE TO DETERMINE                                                                 343762
FAILING TO YIELD RIGHT-OF-WAY                                                        96965
FOLLOWING TOO CLOSELY                                                                84963
NOT APPLICABLE                                                                       46641
IMPROPER OVERTAKING/PASSING                                                          43753
FAILING TO REDUCE SPEED TO AVOID CRASH                                               36967
IMPROPER BACKING                                                                     34175
IMPROPER LANE USAGE                                                                  31351
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  29810
IMPROPER TURNING/NO SIGNAL                                                           29404
DISREGARDING TRAFFIC SIGNALS                                      

In [59]:
# removing the incidents whereby the causes are unknown or were unable to be determined
utd = data_no_admin[data_no_admin['PRIM_CONTRIBUTORY_CAUSE'] == 'UNABLE TO DETERMINE']
utd['SEC_CONTRIBUTORY_CAUSE'].value_counts()
#utd - unable to determine

SEC_CONTRIBUTORY_CAUSE
UNABLE TO DETERMINE                                                                 202690
NOT APPLICABLE                                                                      132483
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                   1864
FAILING TO REDUCE SPEED TO AVOID CRASH                                                 971
WEATHER                                                                                796
IMPROPER LANE USAGE                                                                    675
FOLLOWING TOO CLOSELY                                                                  663
FAILING TO YIELD RIGHT-OF-WAY                                                          662
HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)                                        378
PHYSICAL CONDITION OF DRIVER                                                           357
IMPROPER OVERTAKING/PASSING                                        

In [60]:
n_a = data_no_admin[data_no_admin['PRIM_CONTRIBUTORY_CAUSE'] == 'NOT APPLICABLE']
n_a['SEC_CONTRIBUTORY_CAUSE'].value_counts()
#n_a - not applicable

SEC_CONTRIBUTORY_CAUSE
NOT APPLICABLE                                                                      43527
UNABLE TO DETERMINE                                                                  1947
FAILING TO REDUCE SPEED TO AVOID CRASH                                                124
FAILING TO YIELD RIGHT-OF-WAY                                                         112
WEATHER                                                                                96
FOLLOWING TOO CLOSELY                                                                  93
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                    92
IMPROPER OVERTAKING/PASSING                                                            75
IMPROPER BACKING                                                                       69
IMPROPER LANE USAGE                                                                    63
IMPROPER TURNING/NO SIGNAL                                                   

In [61]:
test_data = data_no_admin.copy()
test_data.head()

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,09/05/2023 07:05:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DUSK,ANGLE,"FIVE POINT, OR MORE",STRAIGHT AND LEVEL,...,2.0,0.0,2.0,0.0,19,3,9,NaN,NaN,NaN
1,2675c13fd0f474d730a5b780968b3cafc7c12d7adb661f...,09/22/2023 06:45:00 PM,50,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,18,6,9,NaN,NaN,NaN
2,5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4...,07/29/2023 02:45:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,0.0,0.0,1.0,0.0,14,7,7,41.85412,-87.665902,POINT (-87.665902342962 41.854120262952)
3,7ebf015016f83d09b321afd671a836d6b148330535d5df...,08/09/2023 11:00:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,0.0,0.0,2.0,0.0,23,4,8,NaN,NaN,NaN
4,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,08/18/2023 12:50:00 PM,15,OTHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,OTHER,STRAIGHT AND LEVEL,...,1.0,0.0,1.0,0.0,12,6,8,NaN,NaN,NaN


In [62]:
test_data['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
UNABLE TO DETERMINE                                                                 343762
FAILING TO YIELD RIGHT-OF-WAY                                                        96965
FOLLOWING TOO CLOSELY                                                                84963
NOT APPLICABLE                                                                       46641
IMPROPER OVERTAKING/PASSING                                                          43753
FAILING TO REDUCE SPEED TO AVOID CRASH                                               36967
IMPROPER BACKING                                                                     34175
IMPROPER LANE USAGE                                                                  31351
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  29810
IMPROPER TURNING/NO SIGNAL                                                           29404
DISREGARDING TRAFFIC SIGNALS                                      

In [63]:
# Iterate over the rows where the primary cause wasn't determined and replace with a secondary cause
for index, row in test_data.iterrows(): 
    if test_data.loc[index,'PRIM_CONTRIBUTORY_CAUSE'] == 'UNABLE TO DETERMINE':
        if (test_data.loc[index,'SEC_CONTRIBUTORY_CAUSE'] != 'UNABLE TO DETERMINE') & (test_data.loc[index,'SEC_CONTRIBUTORY_CAUSE'] != 'NOT APPLICABLE'):
            test_data.loc[index,'PRIM_CONTRIBUTORY_CAUSE'] = test_data.loc[index,'SEC_CONTRIBUTORY_CAUSE']

In [64]:
test_data['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
UNABLE TO DETERMINE                                                                 335173
FAILING TO YIELD RIGHT-OF-WAY                                                        97627
FOLLOWING TOO CLOSELY                                                                85626
NOT APPLICABLE                                                                       46641
IMPROPER OVERTAKING/PASSING                                                          44054
FAILING TO REDUCE SPEED TO AVOID CRASH                                               37938
IMPROPER BACKING                                                                     34468
IMPROPER LANE USAGE                                                                  32026
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  31674
IMPROPER TURNING/NO SIGNAL                                                           29568
DISREGARDING TRAFFIC SIGNALS                                      

In [65]:
for index, row in test_data.iterrows(): 
    if test_data.loc[index,'PRIM_CONTRIBUTORY_CAUSE'] == 'NOT APPLICABLE':
        if (test_data.loc[index,'SEC_CONTRIBUTORY_CAUSE'] != 'UNABLE TO DETERMINE') & (test_data.loc[index,'SEC_CONTRIBUTORY_CAUSE'] != 'NOT APPLICABLE'):
            test_data.loc[index,'PRIM_CONTRIBUTORY_CAUSE'] = test_data.loc[index,'SEC_CONTRIBUTORY_CAUSE']

In [66]:
test_data['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
UNABLE TO DETERMINE                                                                 335173
FAILING TO YIELD RIGHT-OF-WAY                                                        97739
FOLLOWING TOO CLOSELY                                                                85719
NOT APPLICABLE                                                                       45474
IMPROPER OVERTAKING/PASSING                                                          44129
FAILING TO REDUCE SPEED TO AVOID CRASH                                               38062
IMPROPER BACKING                                                                     34537
IMPROPER LANE USAGE                                                                  32089
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  31766
IMPROPER TURNING/NO SIGNAL                                                           29625
DISREGARDING TRAFFIC SIGNALS                                      

In [67]:
primary = test_data.drop(['SEC_CONTRIBUTORY_CAUSE'],axis = 1)

In [68]:
primary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879740 entries, 0 to 879739
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                879740 non-null  object 
 1   CRASH_DATE                     879740 non-null  object 
 2   POSTED_SPEED_LIMIT             879740 non-null  int64  
 3   TRAFFIC_CONTROL_DEVICE         879740 non-null  object 
 4   DEVICE_CONDITION               879740 non-null  object 
 5   WEATHER_CONDITION              879740 non-null  object 
 6   LIGHTING_CONDITION             879740 non-null  object 
 7   FIRST_CRASH_TYPE               879740 non-null  object 
 8   TRAFFICWAY_TYPE                879740 non-null  object 
 9   ALIGNMENT                      879740 non-null  object 
 10  ROADWAY_SURFACE_COND           879740 non-null  object 
 11  ROAD_DEFECT                    879740 non-null  object 
 12  CRASH_TYPE                    

In [69]:
known_causes = primary.loc[(primary['PRIM_CONTRIBUTORY_CAUSE'] != 'NOT APPLICABLE')]
known_causes = known_causes.loc[(primary['PRIM_CONTRIBUTORY_CAUSE'] != 'UNABLE TO DETERMINE')]
known_causes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499093 entries, 1 to 879739
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                499093 non-null  object 
 1   CRASH_DATE                     499093 non-null  object 
 2   POSTED_SPEED_LIMIT             499093 non-null  int64  
 3   TRAFFIC_CONTROL_DEVICE         499093 non-null  object 
 4   DEVICE_CONDITION               499093 non-null  object 
 5   WEATHER_CONDITION              499093 non-null  object 
 6   LIGHTING_CONDITION             499093 non-null  object 
 7   FIRST_CRASH_TYPE               499093 non-null  object 
 8   TRAFFICWAY_TYPE                499093 non-null  object 
 9   ALIGNMENT                      499093 non-null  object 
 10  ROADWAY_SURFACE_COND           499093 non-null  object 
 11  ROAD_DEFECT                    499093 non-null  object 
 12  CRASH_TYPE                     4990

In [70]:
known_causes['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
FAILING TO YIELD RIGHT-OF-WAY                                                       97739
FOLLOWING TOO CLOSELY                                                               85719
IMPROPER OVERTAKING/PASSING                                                         44129
FAILING TO REDUCE SPEED TO AVOID CRASH                                              38062
IMPROPER BACKING                                                                    34537
IMPROPER LANE USAGE                                                                 32089
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                 31766
IMPROPER TURNING/NO SIGNAL                                                          29625
DISREGARDING TRAFFIC SIGNALS                                                        17330
WEATHER                                                                             13622
OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MA

In [71]:
known_causes.info

<bound method DataFrame.info of                                           CRASH_RECORD_ID  \
1       2675c13fd0f474d730a5b780968b3cafc7c12d7adb661f...   
2       5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4...   
3       7ebf015016f83d09b321afd671a836d6b148330535d5df...   
4       6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...   
8       36360857c079418cba1b1d70cf653595bbfb4566de8fcb...   
...                                                   ...   
879731  5914c45ccb3f73c5ad90016bbbc05e4d6ed245dc2ba4e5...   
879733  45d62ddd4e4b5572d253cc072b3927afc15489728f1ba8...   
879736  cb0ca76762ca65feacb1f6513312f96f2a05629bd8a866...   
879737  ede74fa6fd507334cd0415d3d53deb535d27d23523cd38...   
879739  e88a7d78baea3f2de14f0042ce24937d03ca58553c7102...   

                    CRASH_DATE  POSTED_SPEED_LIMIT TRAFFIC_CONTROL_DEVICE  \
1       09/22/2023 06:45:00 PM                  50            NO CONTROLS   
2       07/29/2023 02:45:00 PM                  30         TRAFFIC SIGNAL   
3   

In [72]:
known_causes = known_causes.dropna()
known_causes.info

<bound method DataFrame.info of                                           CRASH_RECORD_ID  \
2       5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4...   
46      c107aea0fb8320ca5e7676759af9399cffe67f3ac8e429...   
56      db61137bc4bb19b84665da12c4c367592e874c623f76a1...   
63      8640aa03a86f10df7a8c1dafc03d1551c4a4c0262397f3...   
74      f1779af8b006d956f29723ac415bd54b61ef9acf84e38e...   
...                                                   ...   
879731  5914c45ccb3f73c5ad90016bbbc05e4d6ed245dc2ba4e5...   
879733  45d62ddd4e4b5572d253cc072b3927afc15489728f1ba8...   
879736  cb0ca76762ca65feacb1f6513312f96f2a05629bd8a866...   
879737  ede74fa6fd507334cd0415d3d53deb535d27d23523cd38...   
879739  e88a7d78baea3f2de14f0042ce24937d03ca58553c7102...   

                    CRASH_DATE  POSTED_SPEED_LIMIT TRAFFIC_CONTROL_DEVICE  \
2       07/29/2023 02:45:00 PM                  30         TRAFFIC SIGNAL   
46      09/19/2023 05:40:00 PM                  30            NO CONTROLS   
56  

In [73]:
# categorical variables
for col in known_causes.columns:
    print(known_causes[col].value_counts())
    print("")

CRASH_RECORD_ID
e88a7d78baea3f2de14f0042ce24937d03ca58553c71022c5abc84cfabfd5f6db42fabfe34751eaa13e6c617a725ab8f42af877c382a4abdab0709b99deccf8a    1
5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4106558b34b8a6d2b81af02cf91b576ecd7ced08ffd10fcfd940a84f7613125b89d33636e6075064e22    1
c107aea0fb8320ca5e7676759af9399cffe67f3ac8e4292e448a54ef70cf2196867af4481c6acb568341407b99430956b7a8a30d7193127d31f16891e90eab8d    1
db61137bc4bb19b84665da12c4c367592e874c623f76a1159a998cf6fafa7dd225820a30a5c4dd36ff4a7147d70c85395dd9b0dfaeea511be8cc8161e8148e07    1
8640aa03a86f10df7a8c1dafc03d1551c4a4c0262397f3ab7bccb2b62e75f94132f1f92953d980712fce89d1c374f1a30c239475c880303d9842bba5b8987775    1
                                                                                                                                   ..
a5148efd1daca1a75626ac46aa374cbedc7dc7c39188e6656bd42bf981e8c7e696b99bcff37c41b8444e91049ce86853dd5eaf153a76249f7b560bd6072fa435    1
2535355d96f75047afe866ec848126deede3e85b693b96

In [74]:
categorical_df = known_causes.copy()

In [75]:
# Road_defect to represent 1 if there was defect, 0 for none 
categorical_df['ROADWAY_SURFACE_COND_DRY'] = categorical_df['ROADWAY_SURFACE_COND'].map(lambda x: 1 if ('DRY' in x or 'UNKNOWN' in x ) else 0)
categorical_df = categorical_df.drop(['ROADWAY_SURFACE_COND'], axis = 1)
categorical_df['ROADWAY_SURFACE_COND_DRY'].value_counts()

ROADWAY_SURFACE_COND_DRY
1    400724
0     93981
Name: count, dtype: int64

In [76]:
# Road_defect to represent 1 if there was defect, 0 for none 
categorical_df['ROAD_DEFECT'] = categorical_df['ROAD_DEFECT'].map(lambda x: 0 if ('NO DEFECTS' in x or 'UNKNOWN' in x ) else 1)
categorical_df['ROAD_DEFECT'].value_counts()

ROAD_DEFECT
0    484868
1      9837
Name: count, dtype: int64

In [77]:
# if crash did not result in injury to be represented by 1, 0 if crash resulted in injury and/or tow
categorical_df['CRASH_TYPE_NO_INJURY'] = categorical_df['CRASH_TYPE'].map(lambda x: 1 if ('NO INJURY' in x ) else 0)
categorical_df = categorical_df.drop(['CRASH_TYPE'], axis = 1)
categorical_df['CRASH_TYPE_NO_INJURY'].value_counts()

CRASH_TYPE_NO_INJURY
1    339487
0    155218
Name: count, dtype: int64

In [78]:
categorical_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 494705 entries, 2 to 879739
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                494705 non-null  object 
 1   CRASH_DATE                     494705 non-null  object 
 2   POSTED_SPEED_LIMIT             494705 non-null  int64  
 3   TRAFFIC_CONTROL_DEVICE         494705 non-null  object 
 4   DEVICE_CONDITION               494705 non-null  object 
 5   WEATHER_CONDITION              494705 non-null  object 
 6   LIGHTING_CONDITION             494705 non-null  object 
 7   FIRST_CRASH_TYPE               494705 non-null  object 
 8   TRAFFICWAY_TYPE                494705 non-null  object 
 9   ALIGNMENT                      494705 non-null  object 
 10  ROAD_DEFECT                    494705 non-null  int64  
 11  INTERSECTION_RELATED_I         494705 non-null  int64  
 12  NOT_RIGHT_OF_WAY_I             4947

In [79]:
categorical_df['TRAFFIC_CONTROL_DEVICE'].value_counts()

TRAFFIC_CONTROL_DEVICE
NO CONTROLS                 252429
TRAFFIC SIGNAL              163883
STOP SIGN/FLASHER            59030
UNKNOWN                      10559
OTHER                         3600
YIELD                         1013
LANE USE MARKING               927
OTHER REG. SIGN                738
OTHER WARNING SIGN             536
PEDESTRIAN CROSSING SIGN       459
RAILROAD CROSSING GATE         375
DELINEATORS                    245
FLASHING CONTROL SIGNAL        230
SCHOOL ZONE                    205
POLICE/FLAGMAN                 205
OTHER RAILROAD CROSSING        129
RR CROSSING SIGN                68
NO PASSING                      50
BICYCLE CROSSING SIGN           24
Name: count, dtype: int64

In [80]:
categorical_df['TRAFFIC_CONTROL_DEVICE'] = categorical_df['TRAFFIC_CONTROL_DEVICE'].apply(lambda x: 'SIGNAL' if 'SIGNAL' in x else x)
categorical_df['TRAFFIC_CONTROL_DEVICE'].value_counts()

TRAFFIC_CONTROL_DEVICE
NO CONTROLS                 252429
SIGNAL                      164113
STOP SIGN/FLASHER            59030
UNKNOWN                      10559
OTHER                         3600
YIELD                         1013
LANE USE MARKING               927
OTHER REG. SIGN                738
OTHER WARNING SIGN             536
PEDESTRIAN CROSSING SIGN       459
RAILROAD CROSSING GATE         375
DELINEATORS                    245
POLICE/FLAGMAN                 205
SCHOOL ZONE                    205
OTHER RAILROAD CROSSING        129
RR CROSSING SIGN                68
NO PASSING                      50
BICYCLE CROSSING SIGN           24
Name: count, dtype: int64

In [81]:
categorical_df['DEVICE_CONDITION'].value_counts()

DEVICE_CONDITION
NO CONTROLS                 257734
FUNCTIONING PROPERLY        206362
UNKNOWN                      22153
OTHER                         3927
FUNCTIONING IMPROPERLY        2750
NOT FUNCTIONING               1497
WORN REFLECTIVE MATERIAL       211
MISSING                         71
Name: count, dtype: int64

In [82]:
device_dict = {'NO CONTROLS': 'NO CONTROLS',
               'FUNCTIONING PROPERLY': 'FUNCTIONING PROPERLY',
               'UNKNOWN': 'NO CONTROLS',
               'FUNCTIONING IMPROPERLY':'FUNCTIONING IMPROPERLY/ MISSING',
               'NOT FUNCTIONING':'FUNCTIONING IMPROPERLY/ MISSING',
               'WORN REFLECTIVE MATERIAL': 'FUNCTIONING PROPERLY',
               'MISSING': 'FUNCTIONING IMPROPERLY/ MISSING',
               'OTHER': 'FUNCTIONING PROPERLY'}
categorical_df['DEVICE_CONDITION'] = categorical_df['DEVICE_CONDITION'].map(device_dict)
categorical_df['DEVICE_CONDITION'].value_counts()

DEVICE_CONDITION
NO CONTROLS                        279887
FUNCTIONING PROPERLY               210500
FUNCTIONING IMPROPERLY/ MISSING      4318
Name: count, dtype: int64

In [83]:
categorical_df['FIRST_CRASH_TYPE'].value_counts()

FIRST_CRASH_TYPE
REAR END                        124024
TURNING                          87834
SIDESWIPE SAME DIRECTION         80205
PARKED MOTOR VEHICLE             71533
ANGLE                            64980
FIXED OBJECT                     19865
PEDESTRIAN                       11395
PEDALCYCLIST                      7892
SIDESWIPE OPPOSITE DIRECTION      6768
REAR TO FRONT                     5676
HEAD ON                           4853
OTHER OBJECT                      3542
REAR TO SIDE                      3333
OTHER NONCOLLISION                1193
REAR TO REAR                       881
ANIMAL                             446
OVERTURNED                         254
TRAIN                               31
Name: count, dtype: int64

In [84]:
categorical_df['FIRST_CRASH_TYPE'] = categorical_df['FIRST_CRASH_TYPE'].apply(lambda x: 'PARKED CAR/OBJECT' if ('PARKED' in x or 'OBJECT' in x) else x)
categorical_df['FIRST_CRASH_TYPE'].value_counts()

FIRST_CRASH_TYPE
REAR END                        124024
PARKED CAR/OBJECT                94940
TURNING                          87834
SIDESWIPE SAME DIRECTION         80205
ANGLE                            64980
PEDESTRIAN                       11395
PEDALCYCLIST                      7892
SIDESWIPE OPPOSITE DIRECTION      6768
REAR TO FRONT                     5676
HEAD ON                           4853
REAR TO SIDE                      3333
OTHER NONCOLLISION                1193
REAR TO REAR                       881
ANIMAL                             446
OVERTURNED                         254
TRAIN                               31
Name: count, dtype: int64

In [85]:
categorical_df['WEATHER_CONDITION'].value_counts()

WEATHER_CONDITION
CLEAR                       396705
RAIN                         47464
SNOW                         17946
CLOUDY/OVERCAST              16064
UNKNOWN                      12017
OTHER                         1502
FREEZING RAIN/DRIZZLE         1107
FOG/SMOKE/HAZE                 858
SLEET/HAIL                     672
BLOWING SNOW                   269
SEVERE CROSS WIND GATE          98
BLOWING SAND, SOIL, DIRT         3
Name: count, dtype: int64

In [86]:
weather_dict = {'CLEAR': 'CLEAR', 'RAIN':'PRECIPITATION','SNOW':'PRECIPITATION',
                'CLOUDY/OVERCAST': 'OTHER', 'OTHER':'CLEAR', 'FREEZING RAIN/DRIZZLE':'PRECIPITATION',
                'OTHER':'OTHER', 'FOG/SMOKE/HAZE':'OTHER', 'SLEET/HAIL':'PRECIPITATION', 'BLOWING SNOW':'PRECIPITATION',
                'SEVERE CROSS WIND GATE':'OTHER', 'UNKNOWN': 'CLEAR'}

categorical_df['WEATHER_CONDITION'] = categorical_df['WEATHER_CONDITION'].map(weather_dict)
categorical_df['WEATHER_CONDITION'].value_counts()

WEATHER_CONDITION
CLEAR            408722
PRECIPITATION     67458
OTHER             18522
Name: count, dtype: int64

In [87]:
categorical_df['MOST_SEVERE_INJURY'].value_counts()

MOST_SEVERE_INJURY
NO INDICATION OF INJURY     411381
NONINCAPACITATING INJURY     47273
REPORTED, NOT EVIDENT        25498
INCAPACITATING INJURY         9991
FATAL                          562
Name: count, dtype: int64

In [88]:
injury_dict = {'NO INDICATION OF INJURY' : 'NO INJURY', 'NONINCAPACITATING INJURY': 'MINOR INJURIES',
              'REPORTED, NOT EVIDENT': 'MINOR INJURIES', 'INCAPACITATING INJURY' : 'MAJOR INJURIES',
              'FATAL': 'FATAL'}

categorical_df['MOST_SEVERE_INJURY'] = categorical_df['MOST_SEVERE_INJURY'].map(injury_dict)
categorical_df['MOST_SEVERE_INJURY'].value_counts()

MOST_SEVERE_INJURY
NO INJURY         411381
MINOR INJURIES     72771
MAJOR INJURIES      9991
FATAL                562
Name: count, dtype: int64

In [89]:
categorical_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 494705 entries, 2 to 879739
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                494705 non-null  object 
 1   CRASH_DATE                     494705 non-null  object 
 2   POSTED_SPEED_LIMIT             494705 non-null  int64  
 3   TRAFFIC_CONTROL_DEVICE         494705 non-null  object 
 4   DEVICE_CONDITION               494705 non-null  object 
 5   WEATHER_CONDITION              494702 non-null  object 
 6   LIGHTING_CONDITION             494705 non-null  object 
 7   FIRST_CRASH_TYPE               494705 non-null  object 
 8   TRAFFICWAY_TYPE                494705 non-null  object 
 9   ALIGNMENT                      494705 non-null  object 
 10  ROAD_DEFECT                    494705 non-null  int64  
 11  INTERSECTION_RELATED_I         494705 non-null  int64  
 12  NOT_RIGHT_OF_WAY_I             4947

In [90]:
# Classifying Promary Cause 
primary_cause = categorical_df.copy()

In [91]:
primary_cause['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
FAILING TO YIELD RIGHT-OF-WAY                                                       96982
FOLLOWING TOO CLOSELY                                                               85002
IMPROPER OVERTAKING/PASSING                                                         43782
FAILING TO REDUCE SPEED TO AVOID CRASH                                              37817
IMPROPER BACKING                                                                    34229
IMPROPER LANE USAGE                                                                 31662
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                 31438
IMPROPER TURNING/NO SIGNAL                                                          29419
DISREGARDING TRAFFIC SIGNALS                                                        17204
WEATHER                                                                             13325
OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MA

In [92]:
primary_cause['PRIM_CONTRIBUTORY_CAUSE'] = primary_cause['PRIM_CONTRIBUTORY_CAUSE'].apply(lambda x: 'EXTERNAL FACTORS' if 'ANIMAL' in x else x)
primary_cause['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
FAILING TO YIELD RIGHT-OF-WAY                                                       96982
FOLLOWING TOO CLOSELY                                                               85002
IMPROPER OVERTAKING/PASSING                                                         43782
FAILING TO REDUCE SPEED TO AVOID CRASH                                              37817
IMPROPER BACKING                                                                    34229
IMPROPER LANE USAGE                                                                 31662
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                 31438
IMPROPER TURNING/NO SIGNAL                                                          29419
DISREGARDING TRAFFIC SIGNALS                                                        17204
WEATHER                                                                             13325
OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MA

In [93]:
primary_cause['PRIM_CONTRIBUTORY_CAUSE'] = primary_cause['PRIM_CONTRIBUTORY_CAUSE'].apply(lambda x: 'EXTERNAL FACTORS' if 'ROAD' in x else x)
primary_cause['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
FAILING TO YIELD RIGHT-OF-WAY                                                       96982
FOLLOWING TOO CLOSELY                                                               85002
IMPROPER OVERTAKING/PASSING                                                         43782
FAILING TO REDUCE SPEED TO AVOID CRASH                                              37817
IMPROPER BACKING                                                                    34229
IMPROPER LANE USAGE                                                                 31662
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                 31438
IMPROPER TURNING/NO SIGNAL                                                          29419
DISREGARDING TRAFFIC SIGNALS                                                        17204
WEATHER                                                                             13325
OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MA

In [94]:
primary_cause['PRIM_CONTRIBUTORY_CAUSE'] = primary_cause['PRIM_CONTRIBUTORY_CAUSE'].apply(lambda x: 'IMPROPER' if 'IMPROPER' in x else x)
primary_cause['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
IMPROPER                                                                            139092
FAILING TO YIELD RIGHT-OF-WAY                                                        96982
FOLLOWING TOO CLOSELY                                                                85002
FAILING TO REDUCE SPEED TO AVOID CRASH                                               37817
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  31438
DISREGARDING TRAFFIC SIGNALS                                                         17204
WEATHER                                                                              13325
OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER     11228
DISREGARDING STOP SIGN                                                                9475
EXTERNAL FACTORS                                                                      7619
DISTRACTION - FROM INSIDE VEHICLE                                 

In [95]:
primary_cause['PRIM_CONTRIBUTORY_CAUSE'] = primary_cause['PRIM_CONTRIBUTORY_CAUSE'].apply(lambda x: 'DISREGARDING SIGNS/SIGNALS' if 'DISREGARDING' in x else x)
primary_cause['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
IMPROPER                                                                            139092
FAILING TO YIELD RIGHT-OF-WAY                                                        96982
FOLLOWING TOO CLOSELY                                                                85002
FAILING TO REDUCE SPEED TO AVOID CRASH                                               37817
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  31438
DISREGARDING SIGNS/SIGNALS                                                           28895
WEATHER                                                                              13325
OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER     11228
EXTERNAL FACTORS                                                                      7619
DISTRACTION - FROM INSIDE VEHICLE                                                     6042
PHYSICAL CONDITION OF DRIVER                                      

In [96]:
reckless_dict = {'TEXTING': 'RECKLESS BEHAVIOR',
                 'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)': 'RECKLESS BEHAVIOR',
                  'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)':'RECKLESS BEHAVIOR',
                 'CELL PHONE USE OTHER THAN TEXTING': 'RECKLESS BEHAVIOR',
                'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)': 'RECKLESS BEHAVIOR',
                'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER' : 'RECKLESS BEHAVIOR',
                'FOLLOWING TOO CLOSELY': 'RECKLESS BEHAVIOR',
                'PASSING STOPPED SCHOOL BUS' : 'RECKLESS BEHAVIOR',
                'PHYSICAL CONDITION OF DRIVER': 'RECKLESS BEHAVIOR',
                'DRIVING ON WRONG SIDE/WRONG WAY':'RECKLESS BEHAVIOR'}

primary_cause['PRIM_CONTRIBUTORY_CAUSE'] = primary_cause['PRIM_CONTRIBUTORY_CAUSE'].map(reckless_dict).fillna(primary_cause['PRIM_CONTRIBUTORY_CAUSE'])
primary_cause['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
IMPROPER                                                139092
RECKLESS BEHAVIOR                                       113982
FAILING TO YIELD RIGHT-OF-WAY                            96982
FAILING TO REDUCE SPEED TO AVOID CRASH                   37817
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                      31438
DISREGARDING SIGNS/SIGNALS                               28895
WEATHER                                                  13325
EXTERNAL FACTORS                                          7619
DISTRACTION - FROM INSIDE VEHICLE                         6042
EQUIPMENT - VEHICLE CONDITION                             5478
VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)      5124
DISTRACTION - FROM OUTSIDE VEHICLE                        3612
EXCEEDING AUTHORIZED SPEED LIMIT                          2024
EXCEEDING SAFE SPEED FOR CONDITIONS                       1711
TURNING RIGHT ON RED                                       681
RELATED TO BUS STOP            

In [97]:
external = {'OBSTRUCTED CROSSWALKS': 'EXTERNAL FACTORS',
                 'DISTRACTION - FROM OUTSIDE VEHICLE': 'EXTERNAL FACTORS',
                 'EQUIPMENT - VEHICLE CONDITION': 'EXTERNAL FACTORS',
                 'DISTRACTION - FROM INSIDE VEHICLE': 'EXTERNAL FACTORS',
                 'WEATHER': 'EXTERNAL FACTORS',
                'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)': 'EXTERNAL FACTORS',
                'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE':'EXTERNAL FACTORS',
                'BICYCLE ADVANCING LEGALLY ON RED LIGHT' :'EXTERNAL FACTORS',
                'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT' :'EXTERNAL FACTORS'}


primary_cause['PRIM_CONTRIBUTORY_CAUSE'] = primary_cause['PRIM_CONTRIBUTORY_CAUSE'].map(external).fillna(primary_cause['PRIM_CONTRIBUTORY_CAUSE'])
primary_cause['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
IMPROPER                                  139092
RECKLESS BEHAVIOR                         113982
FAILING TO YIELD RIGHT-OF-WAY              96982
EXTERNAL FACTORS                           72969
FAILING TO REDUCE SPEED TO AVOID CRASH     37817
DISREGARDING SIGNS/SIGNALS                 28895
EXCEEDING AUTHORIZED SPEED LIMIT            2024
EXCEEDING SAFE SPEED FOR CONDITIONS         1711
TURNING RIGHT ON RED                         681
RELATED TO BUS STOP                          552
Name: count, dtype: int64

In [98]:
full_dict = {'IMPROPER' : 'IMPROPER/AGGRESSIVE DRIVING',
             'FAILING TO YIELD RIGHT-OF-WAY' : 'IMPROPER/AGGRESSIVE DRIVING',
             'EXTERNAL FACTORS': 'EXTERNAL FACTORS/ OTHER',
             'FAILING TO REDUCE SPEED TO AVOID CRASH': 'IMPROPER/AGGRESSIVE DRIVING',
             'DISREGARDING SIGNS/SIGNALS': 'RECKLESS BEHAVIOR',
             'RELATED TO BUS STOP': 'IMPROPER/AGGRESSIVE DRIVING',
             'TURNING RIGHT ON RED': 'RECKLESS BEHAVIOR'}

primary_cause['PRIM_CONTRIBUTORY_CAUSE'] = primary_cause['PRIM_CONTRIBUTORY_CAUSE'].map(full_dict).fillna(primary_cause['PRIM_CONTRIBUTORY_CAUSE'])
primary_cause['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
IMPROPER/AGGRESSIVE DRIVING            274443
RECKLESS BEHAVIOR                      143558
EXTERNAL FACTORS/ OTHER                 72969
EXCEEDING AUTHORIZED SPEED LIMIT         2024
EXCEEDING SAFE SPEED FOR CONDITIONS      1711
Name: count, dtype: int64

In [99]:
primary_cause['PRIM_CONTRIBUTORY_CAUSE'].value_counts(normalize = True)

PRIM_CONTRIBUTORY_CAUSE
IMPROPER/AGGRESSIVE DRIVING            0.554761
RECKLESS BEHAVIOR                      0.290189
EXTERNAL FACTORS/ OTHER                0.147500
EXCEEDING AUTHORIZED SPEED LIMIT       0.004091
EXCEEDING SAFE SPEED FOR CONDITIONS    0.003459
Name: proportion, dtype: float64

In [100]:
primary_cause.info()

<class 'pandas.core.frame.DataFrame'>
Index: 494705 entries, 2 to 879739
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                494705 non-null  object 
 1   CRASH_DATE                     494705 non-null  object 
 2   POSTED_SPEED_LIMIT             494705 non-null  int64  
 3   TRAFFIC_CONTROL_DEVICE         494705 non-null  object 
 4   DEVICE_CONDITION               494705 non-null  object 
 5   WEATHER_CONDITION              494702 non-null  object 
 6   LIGHTING_CONDITION             494705 non-null  object 
 7   FIRST_CRASH_TYPE               494705 non-null  object 
 8   TRAFFICWAY_TYPE                494705 non-null  object 
 9   ALIGNMENT                      494705 non-null  object 
 10  ROAD_DEFECT                    494705 non-null  int64  
 11  INTERSECTION_RELATED_I         494705 non-null  int64  
 12  NOT_RIGHT_OF_WAY_I             4947

In [101]:
# Dropping some of the irrelevant columns
primary_cause = primary_cause.drop(columns = ['LOCATION', 'STREET_NAME', 
                                              'STREET_DIRECTION','STREET_NO','BEAT_OF_OCCURRENCE'], axis = 1)

In [102]:
fatal_acc = categorical_df[categorical_df['MOST_SEVERE_INJURY'] == 'FATAL']
fatal_acc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 562 entries, 825 to 879377
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   CRASH_RECORD_ID                562 non-null    object 
 1   CRASH_DATE                     562 non-null    object 
 2   POSTED_SPEED_LIMIT             562 non-null    int64  
 3   TRAFFIC_CONTROL_DEVICE         562 non-null    object 
 4   DEVICE_CONDITION               562 non-null    object 
 5   WEATHER_CONDITION              562 non-null    object 
 6   LIGHTING_CONDITION             562 non-null    object 
 7   FIRST_CRASH_TYPE               562 non-null    object 
 8   TRAFFICWAY_TYPE                562 non-null    object 
 9   ALIGNMENT                      562 non-null    object 
 10  ROAD_DEFECT                    562 non-null    int64  
 11  INTERSECTION_RELATED_I         562 non-null    int64  
 12  NOT_RIGHT_OF_WAY_I             562 non-null    int

In [103]:
print(fatal_acc.columns)


Index(['CRASH_RECORD_ID', 'CRASH_DATE', 'POSTED_SPEED_LIMIT',
       'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE',
       'ALIGNMENT', 'ROAD_DEFECT', 'INTERSECTION_RELATED_I',
       'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE',
       'PRIM_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION',
       'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'DOORING_I', 'WORK_ZONE_I',
       'WORKERS_PRESENT_I', 'NUM_UNITS', 'MOST_SEVERE_INJURY',
       'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING',
       'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
       'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'CRASH_HOUR',
       'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'LATITUDE', 'LONGITUDE', 'LOCATION',
       'ROADWAY_SURFACE_COND_DRY', 'CRASH_TYPE_NO_INJURY'],
      dtype='object')


In [104]:
primary_cause.info()

<class 'pandas.core.frame.DataFrame'>
Index: 494705 entries, 2 to 879739
Data columns (total 35 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                494705 non-null  object 
 1   CRASH_DATE                     494705 non-null  object 
 2   POSTED_SPEED_LIMIT             494705 non-null  int64  
 3   TRAFFIC_CONTROL_DEVICE         494705 non-null  object 
 4   DEVICE_CONDITION               494705 non-null  object 
 5   WEATHER_CONDITION              494702 non-null  object 
 6   LIGHTING_CONDITION             494705 non-null  object 
 7   FIRST_CRASH_TYPE               494705 non-null  object 
 8   TRAFFICWAY_TYPE                494705 non-null  object 
 9   ALIGNMENT                      494705 non-null  object 
 10  ROAD_DEFECT                    494705 non-null  int64  
 11  INTERSECTION_RELATED_I         494705 non-null  int64  
 12  NOT_RIGHT_OF_WAY_I             4947

In [105]:
#selecting the most relevant features
features_c = crash[['CRASH_RECORD_ID','CRASH_TYPE', 'POSTED_SPEED_LIMIT','WEATHER_CONDITION','DAMAGE','LIGHTING_CONDITION','ROAD_DEFECT','PRIM_CONTRIBUTORY_CAUSE','SEC_CONTRIBUTORY_CAUSE','CRASH_DAY_OF_WEEK','CRASH_HOUR','ROADWAY_SURFACE_COND']]

In [106]:
crash_df = features_c.copy()

In [107]:
# merge the 3 datasets
df_merge = pd.merge(crash_df, vehicles_df, on='CRASH_RECORD_ID').reset_index()
df = pd.merge(df_merge, people_df, on='CRASH_RECORD_ID').reset_index()

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4058147 entries, 0 to 4058146
Data columns (total 25 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   level_0                  int64  
 1   index                    int64  
 2   CRASH_RECORD_ID          object 
 3   CRASH_TYPE               object 
 4   POSTED_SPEED_LIMIT       int64  
 5   WEATHER_CONDITION        object 
 6   DAMAGE                   object 
 7   LIGHTING_CONDITION       object 
 8   ROAD_DEFECT              object 
 9   PRIM_CONTRIBUTORY_CAUSE  object 
 10  SEC_CONTRIBUTORY_CAUSE   object 
 11  CRASH_DAY_OF_WEEK        int64  
 12  CRASH_HOUR               int64  
 13  ROADWAY_SURFACE_COND     object 
 14  VEHICLE_TYPE             object 
 15  VEHICLE_USE              object 
 16  VEHICLE_DEFECT           object 
 17  MANEUVER                 object 
 18  UNIT_TYPE                object 
 19  AGE                      float64
 20  INJURY_CLASSIFICATION    object 
 21  PERSON_I

In [109]:
# observing values of crash_hour
df['CRASH_HOUR'].value_counts()

CRASH_HOUR
15    332850
16    332796
17    323579
14    277815
18    264543
13    247382
12    233708
8     209855
11    198139
19    190987
10    172744
9     172182
7     169402
20    150698
21    137085
22    124409
23    104469
0      82385
6      76752
1      69623
2      57904
5      45735
3      44858
4      38247
Name: count, dtype: int64

In [110]:
# creating bins for times 
# 0-6 = Late Night/Early Morning 
# 6-12 = Morning 
# 12-18 = Afternoon/Rush Hour
# 18-23 = Late Evening 
df['TIME_BINS'] = pd.cut(x=df['CRASH_HOUR'], bins = [0,6,12,18,23], 
                         labels = ['Late Night/Early Morning', 
                        'Morning', 'Afternoon/Rush Hour','Late Evening'])
df.head()

,level_0,index,CRASH_RECORD_ID,CRASH_TYPE,POSTED_SPEED_LIMIT,WEATHER_CONDITION,DAMAGE,LIGHTING_CONDITION,ROAD_DEFECT,PRIM_CONTRIBUTORY_CAUSE,...,VEHICLE_DEFECT,MANEUVER,UNIT_TYPE,AGE,INJURY_CLASSIFICATION,PERSON_ID,SEX,PERSON_TYPE,PHYSICAL_CONDITION,TIME_BINS
0,0,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,UNKNOWN,STRAIGHT AHEAD,DRIVER,51.0,NONINCAPACITATING INJURY,O1654630,M,DRIVER,NORMAL,Late Evening
1,1,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,UNKNOWN,STRAIGHT AHEAD,DRIVER,36.0,NO INDICATION OF INJURY,O1654631,F,DRIVER,NORMAL,Late Evening
2,2,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,UNKNOWN,STRAIGHT AHEAD,DRIVER,22.0,NONINCAPACITATING INJURY,P366256,M,PASSENGER,NaN,Late Evening
3,3,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,UNKNOWN,STRAIGHT AHEAD,DRIVER,55.0,INCAPACITATING INJURY,P366257,M,PASSENGER,NaN,Late Evening
4,4,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,UNKNOWN,STRAIGHT AHEAD,DRIVER,46.0,NO INDICATION OF INJURY,P366258,M,PASSENGER,NaN,Late Evening


In [111]:
# creating bins and labels, preview data
df['AGE_BINS'] = pd.cut(x=df['AGE'], bins = [0,15,24,35,55,100], 
                         labels = ['15 & Under', '16-24', 
                        '25-35', '36-55','56+'])
df.head()

,level_0,index,CRASH_RECORD_ID,CRASH_TYPE,POSTED_SPEED_LIMIT,WEATHER_CONDITION,DAMAGE,LIGHTING_CONDITION,ROAD_DEFECT,PRIM_CONTRIBUTORY_CAUSE,...,MANEUVER,UNIT_TYPE,AGE,INJURY_CLASSIFICATION,PERSON_ID,SEX,PERSON_TYPE,PHYSICAL_CONDITION,TIME_BINS,AGE_BINS
0,0,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,STRAIGHT AHEAD,DRIVER,51.0,NONINCAPACITATING INJURY,O1654630,M,DRIVER,NORMAL,Late Evening,36-55
1,1,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,STRAIGHT AHEAD,DRIVER,36.0,NO INDICATION OF INJURY,O1654631,F,DRIVER,NORMAL,Late Evening,36-55
2,2,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,STRAIGHT AHEAD,DRIVER,22.0,NONINCAPACITATING INJURY,P366256,M,PASSENGER,NaN,Late Evening,16-24
3,3,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,STRAIGHT AHEAD,DRIVER,55.0,INCAPACITATING INJURY,P366257,M,PASSENGER,NaN,Late Evening,36-55
4,4,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,STRAIGHT AHEAD,DRIVER,46.0,NO INDICATION OF INJURY,P366258,M,PASSENGER,NaN,Late Evening,36-55


In [112]:
df['POSTED_SPEED_LIMIT'].value_counts()

POSTED_SPEED_LIMIT
30    3096311
35     289078
25     229723
20     136351
15     106180
10      67422
40      46828
45      31499
0       29244
5       16405
55       4354
50       1377
3        1047
39        424
9         400
99        291
60        262
24        128
1         125
65         90
2          89
32         79
33         66
34         66
36         46
11         40
26         39
6          26
7          25
14         18
70         15
31         14
22         12
29         11
12          9
23          8
18          8
38          8
8           8
4           7
44          6
63          4
62          2
49          1
46          1
Name: count, dtype: int64

In [113]:
# creating bins and label, previewing data 
df['POSTED_SPEED'] = pd.cut(x=df['POSTED_SPEED_LIMIT'], bins = [0,15,25,40,75], 
                         labels = ['0-15', '16-25', 
                        '26-40', '41+'])
df.head()

,level_0,index,CRASH_RECORD_ID,CRASH_TYPE,POSTED_SPEED_LIMIT,WEATHER_CONDITION,DAMAGE,LIGHTING_CONDITION,ROAD_DEFECT,PRIM_CONTRIBUTORY_CAUSE,...,UNIT_TYPE,AGE,INJURY_CLASSIFICATION,PERSON_ID,SEX,PERSON_TYPE,PHYSICAL_CONDITION,TIME_BINS,AGE_BINS,POSTED_SPEED
0,0,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,DRIVER,51.0,NONINCAPACITATING INJURY,O1654630,M,DRIVER,NORMAL,Late Evening,36-55,26-40
1,1,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,DRIVER,36.0,NO INDICATION OF INJURY,O1654631,F,DRIVER,NORMAL,Late Evening,36-55,26-40
2,2,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,DRIVER,22.0,NONINCAPACITATING INJURY,P366256,M,PASSENGER,NaN,Late Evening,16-24,26-40
3,3,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,DRIVER,55.0,INCAPACITATING INJURY,P366257,M,PASSENGER,NaN,Late Evening,36-55,26-40
4,4,0,23a79931ef555d54118f64dc9be2cf2dbf59636ce253f7...,INJURY AND / OR TOW DUE TO CRASH,30,CLEAR,"OVER $1,500",DUSK,NO DEFECTS,UNABLE TO DETERMINE,...,DRIVER,46.0,NO INDICATION OF INJURY,P366258,M,PASSENGER,NaN,Late Evening,36-55,26-40


In [114]:
df['WEATHER_CONDITION'].value_counts()

WEATHER_CONDITION
CLEAR                       3230870
RAIN                         371320
UNKNOWN                      173627
SNOW                         124017
CLOUDY/OVERCAST              123872
OTHER                         12582
FREEZING RAIN/DRIZZLE          8142
FOG/SMOKE/HAZE                 6291
SLEET/HAIL                     4739
BLOWING SNOW                   2120
SEVERE CROSS WIND GATE          541
BLOWING SAND, SOIL, DIRT         26
Name: count, dtype: int64

In [115]:
# defining dictionary map and remapping
weather_map = {'CLEAR': 'CLEAR',
                'RAIN': 'RAIN/CLOUDY/OTHER',
                'CLOUDY/OVERCAST': 'RAIN/CLOUDY/OTHER',
                'UNKNOWN': 'RAIN/CLOUDY/OTHER',
                'OTHER ': 'RAIN/CLOUDY/OTHER'}

df.WEATHER_CONDITION = df.WEATHER_CONDITION.map(weather_map)
df.WEATHER_CONDITION.value_counts()

WEATHER_CONDITION
CLEAR                3230870
RAIN/CLOUDY/OTHER     668819
Name: count, dtype: int64

In [116]:
df['ROAD_DEFECT'].value_counts()

ROAD_DEFECT
NO DEFECTS           3282165
UNKNOWN               714002
OTHER                  19625
WORN SURFACE           16872
RUT, HOLES             15958
SHOULDER DEFECT         6844
DEBRIS ON ROADWAY       2681
Name: count, dtype: int64

In [117]:
# defining dictionary map and remapping
defect_dict = {'NO DEFECTS': 'NO DEFECTS',
                'UNKNOWN': 'UNKNOWN/OTHER',
                'OTHER': 'UNKNOWN/OTHER',
                'SHOULDER DEFECT': 'UNKNOWN/OTHER'}

df.ROAD_DEFECT = df.ROAD_DEFECT.map(defect_dict)
df.ROAD_DEFECT.value_counts()

ROAD_DEFECT
NO DEFECTS       3282165
UNKNOWN/OTHER     740471
Name: count, dtype: int64

In [118]:
df['VEHICLE_TYPE'].value_counts()

VEHICLE_TYPE
PASSENGER                                 2555181
SPORT UTILITY VEHICLE (SUV)                577280
UNKNOWN/NA                                 270486
VAN/MINI-VAN                               192617
PICKUP                                     127096
TRUCK - SINGLE UNIT                         66428
BUS OVER 15 PASS.                           54172
OTHER                                       45542
TRACTOR W/ SEMI-TRAILER                     31679
BUS UP TO 15 PASS.                          12879
MOTORCYCLE (OVER 150CC)                      8934
SINGLE UNIT TRUCK WITH TRAILER               5583
OTHER VEHICLE WITH TRAILER                   5474
TRACTOR W/O SEMI-TRAILER                     4549
MOPED OR MOTORIZED BICYCLE                   1342
AUTOCYCLE                                    1123
MOTOR DRIVEN CYCLE                            624
ALL-TERRAIN VEHICLE (ATV)                     391
FARM EQUIPMENT                                174
3-WHEELED MOTORCYCLE (2 REAR WHEELS) 

In [119]:
# defining dictionary map and remapping
vehicle_dict = {'PASSENGER': 'PASSENGER',
                'SPORT UTILITY VEHICLE (SUV)': 'SUV/VAN/PICKUP',
                'UNKNOWN/NA': 'UNKNOWN/NA',
                'VAN/MINI-VAN': 'SUV/VAN/PICKUP',
                'PICKUP': 'SUV/VAN/PICKUP',
                'OTHER': 'OTHER',
                'TRACTOR W/ SEMI-TRAILER': 'BUS/TRUCK/TRAILER',
                'TRUCK - SINGLE UNIT': 'BUS/TRUCK/TRAILER',
                'BUS UP TO 15 PASS.': 'BUS/TRUCK/TRAILER',
                'BUS OVER 15 PASS.': 'BUS/TRUCK/TRAILER',
                'SINGLE UNIT TRUCK WITH TRAILER': 'BUS/TRUCK/TRAILER',
                'MOTORCYCLE (OVER 150CC)': 'OTHER',
                'TRACTOR W/O SEMI-TRAILER': 'OTHER',
                'OTHER VEHICLE WITH TRAILER': 'OTHER'}

df.VEHICLE_TYPE = df.VEHICLE_TYPE.map(vehicle_dict)
df.VEHICLE_TYPE.value_counts()

VEHICLE_TYPE
PASSENGER            2555181
SUV/VAN/PICKUP        896993
UNKNOWN/NA            270486
BUS/TRUCK/TRAILER     170741
OTHER                  64499
Name: count, dtype: int64

In [120]:
df ['PERSON_TYPE'].value_counts()

PERSON_TYPE
DRIVER                 3144729
PASSENGER               829459
PEDESTRIAN               50105
BICYCLE                  29427
NON-MOTOR VEHICLE         3591
NON-CONTACT VEHICLE        836
Name: count, dtype: int64

In [121]:
driver_dict ={'DRIVER': 'DRIVER',
             'PASSENGER':'OTHER',
             'PEDESTRIAN':'OTHER',
             'BICYCLE':'OTHER',
             'NON-MOTOR VEHICLE':'OTHER',
             'NON-CONTACT VEHICLE': 'OTHER'}
df.PERSON_TYPE = df.PERSON_TYPE.map(driver_dict)
df.PERSON_TYPE.value_counts()

PERSON_TYPE
DRIVER    3144729
OTHER      913418
Name: count, dtype: int64

In [122]:
df['UNIT_TYPE'].value_counts()

UNIT_TYPE
DRIVER                 3631075
PARKED                  314912
PEDESTRIAN               53855
BICYCLE                  31526
DRIVERLESS               17560
NON-MOTOR VEHICLE         3726
NON-CONTACT VEHICLE        838
DISABLED VEHICLE           383
EQUESTRIAN                   9
Name: count, dtype: int64

In [123]:
df['PHYSICAL_CONDITION'].value_counts()

PHYSICAL_CONDITION
NORMAL                          2091790
UNKNOWN                         1068986
IMPAIRED - ALCOHOL                14771
REMOVED BY EMS                    12301
OTHER                              9550
FATIGUED/ASLEEP                    8654
EMOTIONAL                          8564
ILLNESS/FAINTED                    2973
HAD BEEN DRINKING                  2495
IMPAIRED - DRUGS                   1609
IMPAIRED - ALCOHOL AND DRUGS        953
MEDICATED                           403
Name: count, dtype: int64

In [124]:
dropping = ['CRASH_TYPE','DAMAGE','LIGHTING_CONDITION','PERSON_ID','MANEUVER','UNIT_TYPE','PHYSICAL_CONDITION']
df = df.drop(columns=dropping)

In [125]:
#dropping following columns because we created bins 
drop = ['POSTED_SPEED_LIMIT', 'CRASH_HOUR', 'AGE']
df = df.drop(columns=drop)
# converting columns to appropriate data types 
df['CRASH_DAY_OF_WEEK'] = df['CRASH_DAY_OF_WEEK'].astype('str')

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4058147 entries, 0 to 4058146
Data columns (total 18 columns):
 #   Column                   Dtype   
---  ------                   -----   
 0   level_0                  int64   
 1   index                    int64   
 2   CRASH_RECORD_ID          object  
 3   WEATHER_CONDITION        object  
 4   ROAD_DEFECT              object  
 5   PRIM_CONTRIBUTORY_CAUSE  object  
 6   SEC_CONTRIBUTORY_CAUSE   object  
 7   CRASH_DAY_OF_WEEK        object  
 8   ROADWAY_SURFACE_COND     object  
 9   VEHICLE_TYPE             object  
 10  VEHICLE_USE              object  
 11  VEHICLE_DEFECT           object  
 12  INJURY_CLASSIFICATION    object  
 13  SEX                      object  
 14  PERSON_TYPE              object  
 15  TIME_BINS                category
 16  AGE_BINS                 category
 17  POSTED_SPEED             category
dtypes: category(3), int64(2), object(13)
memory usage: 476.0+ MB


#### MODELLING

In [127]:
## setting X, y for train-test-split
X = df.drop('PRIM_CONTRIBUTORY_CAUSE', axis=1)  
y = df['PRIM_CONTRIBUTORY_CAUSE'] 

In [128]:
# List of categorical columns
categorical_cols = ['WEATHER_CONDITION','ROAD_DEFECT','SEC_CONTRIBUTORY_CAUSE','CRASH_DAY_OF_WEEK','ROADWAY_SURFACE_COND',
                   'VEHICLE_TYPE','VEHICLE_USE','level_0','index','VEHICLE_DEFECT','INJURY_CLASSIFICATION','SEX','PERSON_TYPE',
                   'TIME_BINS','AGE_BINS','POSTED_SPEED']

# Create a pipeline with preprocessing and logistic regression
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values
            ('encoder', OneHotEncoder(handle_unknown='ignore'))   # One-hot encode
        ]), categorical_cols)
    ])

pipe_logistic = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

In [130]:
# train-test-split, test_size = 25%, random_state = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=42)

In [131]:
# Fit and transform on the train data
X_train_transformed = preprocessor.fit_transform(X_train)

# Apply the same transformation to the test data
X_test_transformed = preprocessor.transform(X_test)

X_train_transformed.shape, X_test_transformed.shape

((3043610, 4671399), (1014537, 4671399))

In [135]:
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Keep the encoder for later use if needed

# Now all specified columns should be of float dtype
df[categorical_cols] = df[categorical_cols].astype(float)


In [136]:
# Initiate metrics list 
metrics_list=[]

class Metrics:
    def model_score(self, model, y_true, y_pred):
        # Print classification report, accuracy, precision, recall, f1_score 
        print(classification_report(y_true, y_pred))
        print("Overall accuracy score", accuracy_score(y_true, y_pred))
        print("Overall precision score", precision_score(y_true, y_pred, average='weighted'))
        print("Overall recall score", recall_score(y_true, y_pred, average='weighted'))
        print("Overall F1-score", f1_score(y_true, y_pred, average='weighted'))

        # Print a confusion matrix    
        cnf_matrix = confusion_matrix(y_true, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cnf_matrix, display_labels=model.classes_)
        print('\nConfusion Matrix')       
        return disp.plot()

    # Create a list of model metrics
    def get_metrics(self, model_name, model, y_true, y_pred):  #y_test, X_test, model
        metrics = {}  
        metrics['model_name'] = model_name
        metrics['accuracy'] = accuracy_score(y_true, y_pred)
        metrics['f1 score'] = f1_score(y_true, y_pred, average='weighted')
        metrics['precision'] = precision_score(y_true, y_pred, average='weighted')
        metrics['recall'] = recall_score(y_true, y_pred, average='weighted')
        metrics_list.append(metrics)
        return metrics_list

In [137]:
# Baseline model pipeline
# pipeline for baseline logistic regression
pipe_logistic = Pipeline([('ss', StandardScaler()), 
                    ('lr', LogisticRegression(random_state=42))])

# pipeline for baseline decision tree classification
pipe_decision_tree = Pipeline([('ss', StandardScaler()), 
                    ('tree', DecisionTreeClassifier(random_state=42))])

Model 1 : Logistic Regression

In [138]:
# Fit the logistic regression pipeline to the training data
log_model = pipe_logistic.fit(X_train, y_train)
# Print the accuracy on test set
pipe_logistic.score(X_test, y_test)

ValueError: could not convert string to float: '9653a3c99c25057c1e0fc045f6b62e38d0e63765dea76a730e0a6d1d2b21d690d64528e06b917a3133df3871d5042c3f853dad994a0e631f3645a9af35d3d604'

In [ ]:
# Print model metrics
# Predict target
y_pred = log_model.predict(X_test)

# Create metrics object
score_metrics = Metrics()

# Print classification report, scores, and confusion matrix
score_metrics.model_score(log_model, y_test, y_pred)

In [ ]:
# classification report using function 
evaluate_classification(log_reg,X_train, X_test, y_train, y_test, label = "Logistic Regression")

Model 2 : K- Nearest Neighbors

In [151]:
# instantiate KneighborsClassifier 
knn_clf = KNeighborsClassifier()

In [ ]:
# fit the model 
knn_clf.fit(X_train, y_train)

# predict
y_pred = knn_clf.predict(X_test)

In [ ]:
# classification report using function 
evaluate_classification(knn_clf, X_train, X_test, y_train, y_test, label = 'KNN')

In [ ]:
# create params_grid for KNeighborsClassifier  
params_grid = {'n_neighbors': [3, 5, 7, 9, 11, 21],
              'weights': ['uniform', 'distance'],
              'metric': ['euclidean', 'manhattan']}

# create grid search 
grid = GridSearchCV(knn_clf, params_grid, cv=3)

# fit x_train and y_train to grid 
grid.fit(X_train, y_train)

# observe combination of best params 
grid.best_params_

In [ ]:
# classification report using function
evaluate_classification(grid.best_estimator_, X_train, X_test, y_train, y_test, label='KNN')

Model 3 : XGBoost

In [ ]:
clf = xgb.XGBClassifier(random_state = 42)
clf.fit(X_train, y_train)
training_preds = clf.predict(X_train)
val_preds = clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

In [ ]:
param_grid = {
    "early_stopping_rounds": [10],
    "learning_rate": [0.1],
    'max_depth': [5,6],
    'min_child_weight': [10],
    'subsample': [0.75],
    'n_estimators': [200, 250],
}

In [ ]:
grid_clf = GridSearchCV(clf, param_grid, scoring='accuracy', cv=None, n_jobs=1)
grid_clf.fit(X_train, y_train)

best_parameters = grid_clf.best_params_

print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

training_preds = grid_clf.predict(X_train)
val_preds = grid_clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("")
print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))grid_clf = GridSearchCV(clf, param_grid, scoring='accuracy', cv=None, n_jobs=1)
grid_clf.fit(X_train, y_train)

best_parameters = grid_clf.best_params_

print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

training_preds = grid_clf.predict(X_train)
val_preds = grid_clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("")
print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

In [ ]:
plt.figure(figsize =(10,10))
plot_confusion_matrix(grid_clf, X_train, y_train,
                     cmap=plt.cm.Blues,normalize='true')
plt.xticks(
    rotation=45, 
    horizontalalignment='right',
    fontsize='small'  
)
plt.show()

In [ ]:
plt.figure(figsize =(10,10))
plot_confusion_matrix(grid_clf,X_test, y_test,
                     cmap=plt.cm.Blues,normalize='true')
plt.xticks(
    rotation=45, 
    horizontalalignment='right',
    fontsize='small'
)
plt.show()

In [ ]:
plt.figure(figsize =(10,10))
plot_confusion_matrix(grid_clf, X_test, y_test, values_format = '.0f',
                     cmap=plt.cm.Blues)
plt.xticks(
    rotation=45, 
    horizontalalignment='right',
    fontsize='small'
)
plt.show()